In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1653372706412_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Mock boletos liquidados

In [ ]:
import datetime
import json
from pprint import pprint
from pyspark.sql.functions import lit
import boto3
import pyspark.sql.functions as f 
import sys
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.utils import getResolvedOptions
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType
from awsglue.dynamicframe import DynamicFrame
enviroment = 'des'

In [3]:
import datetime
dt_yesterday_referencia = datetime.datetime.now() - datetime.timedelta(days=1)
year_yesterday_referencia = dt_yesterday_referencia.year
month_yesterday_referencia = str(dt_yesterday_referencia.month).zfill(2)
day_yesterday_referencia= str(dt_yesterday_referencia.day).zfill(2)
date_yesterday_referencia = '%s%s%s' % (year_yesterday_referencia,  month_yesterday_referencia, day_yesterday_referencia)
date_yesterday_referencia_query = '%s%s%s%s%s%s' % (year_yesterday_referencia,'-',month_yesterday_referencia,'-',day_yesterday_referencia, '%')
date_today_processamento  = datetime.datetime.now() - datetime.timedelta(days=0)
year_today_processamento = date_today_processamento.year
month_today_processamento = str(date_today_processamento.month).zfill(2)
day_today_processamento = str(date_today_processamento.day).zfill(2)
date_today_processamento = '%s%s%s' % (year_today_processamento, month_today_processamento, day_today_processamento)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
import datetime
fonte_em_avro = glueContext.create_dynamic_frame_from_options(
connection_type = 's3', 
connection_options = {'paths': ['s3://pessoajur/locais/pastas/boleto.csv']},
format = 'csv',format_options= {'withHeader': True, 'separator':';'})



fonte = fonte_em_avro.toDF()
# fonte.select('*').show(truncate=False)
# fonte.printSchema()

# fonte.select('data.texto_dado_beneficiario_cobranca.numero_agencia', 'data.texto_dado_beneficiario_cobranca.numero_conta_corrente',,
# 'data.texto_dado_beneficiario_cobranca.numero_digito_verificador',   'data.valor_pago_boleto_cobranca' )        

# print('fonte show'),
# fonte.show(),


# #config datas,
# import datetime
dt_yesterday_referencia = datetime.datetime.now() - datetime.timedelta(days=1)
year_yesterday_referencia = dt_yesterday_referencia.year
month_yesterday_referencia = str(dt_yesterday_referencia.month).zfill(2)
day_yesterday_referencia= str(dt_yesterday_referencia.day).zfill(2)
date_yesterday_referencia = '%s%s%s' % (year_yesterday_referencia,  month_yesterday_referencia, day_yesterday_referencia)
date_yesterday_referencia_query = '%s%s%s%s%s%s' % (year_yesterday_referencia,'-',month_yesterday_referencia,'-',day_yesterday_referencia, '%')
date_today_processamento  = datetime.datetime.now() - datetime.timedelta(days=0)
year_today_processamento = date_today_processamento.year
month_today_processamento = str(date_today_processamento.month).zfill(2)
day_today_processamento = str(date_today_processamento.day).zfill(2)
date_today_processamento = '%s%s%s' % (year_today_processamento, month_today_processamento, day_today_processamento)

# from pyspark.sql.functions import lit, col
# # df = fonte.withColumn('year', lit(year_yesterday_referencia))
# #             .withColumn('month', lit(month_yesterday_referencia))
# #             .withColumn('day', lit(day_yesterday_referencia))
# #             .withColumn('type', lit('BONUS_COBRANCA'))
# #             .withColumn('codigo_identificacao_cliente', lit(None))
# #             .withColumn('codigo_segmento', lit(None))
# #             .withColumn('codigo_tipo_pessoa', lit(None))
# #             .withColumn('numero_cadastro_nacional_pessoa_juridica', lit(None))
# #             .withColumn('numero_digito_verificador', lit('1'))
# #             .drop('time'),
# # print('df'),
# # df.show(),
# # dif = DynamicFrame.fromDF(df, glueContext, 'test_nest'),

# # #saida,
# # output_df= '%s%s%s'%('s3://dadosrelacionamento-',environment,
# #                       '/bonus/normalizacao/saida/'),

 
from pyspark.sql.functions import * 
from pyspark.sql.functions import lpad
df = fonte.withColumn('year', lit(year_yesterday_referencia))\
            .withColumn('month', lpad(lit(month_yesterday_referencia),20, '0'))\
            .withColumn('day', lpad(lit(day_yesterday_referencia),20, '0'))\
            .withColumn('codigo_identificacao_cliente', lit(None))\
            .withColumn('codigo_segmento', lit(None))\
            .withColumn('type', lit('BONUS_COBRANCA'))\
            .withColumn('partition_agencia', fonte.numero_agencia.substr(1,1))


df.show()
# df1 = df.withColumn('grad_score_new', lpad(lit(month_yesterday_referencia),5, '0'))
# df1.show()
# # from pyspark.sql import functions as sf
# df.withColumn("month", sf.format_string("%03d","month")).show()
# df.show()
# import pyspark.sql.functions as F
# dfNew = fonte.withColumn('month', F.lpad(month_yesterday_referencia, 4, '0')) 
# df.show()
# df.printSchema()

df.printSchema()
#salvar para criar tabela,
BUCKET = 's3://dadosrelacionamento-'
BUCKET_OUTPUT_DF= '%s%s' % (BUCKET, '2')
output_sumarizador= '%s%s' % ('s3://pessoajur','/bonus/boletos_liquidados/filtrador')
from awsglue.dynamicframe import DynamicFrame
dif = DynamicFrame.fromDF(df, glueContext, 'test_nest')
datasink= glueContext.write_dynamic_frame.from_options(frame = dif,\
connection_type = 's3', \
connection_options = {'path': output_sumarizador,\
    'compression': 'snappy',\
      'partitionKeys': ['year', 'month', 'day', 'type', 'numero_digito_verificador', 'partition_agencia']},
    format='glueparquet',
transformation_ctx = 'datasink')

# # # job.commit()
# #;


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------------------+-------------------------+--------------------------+----+--------------------+--------------------+--------------+-----------------+---------------+------------------+----------------------------------------+----------------------------+
|numero_agencia|numero_conta_corrente|numero_digito_verificador|valor_pago_boleto_cobranca|year|               month|                 day|          type|partition_agencia|codigo_segmento|codigo_tipo_pessoa|numero_cadastro_nacional_pessoa_juridica|codigo_identificacao_cliente|
+--------------+---------------------+-------------------------+--------------------------+----+--------------------+--------------------+--------------+-----------------+---------------+------------------+----------------------------------------+----------------------------+
|           111|                12345|                        0|                        10|2022|00000000000000000005|00000000000000000023|BONUS_COBRANCA|                

In [ ]:
import datetime
import json
from pprint import pprint
from pyspark.sql.functions import lit
import boto3
import pyspark.sql.functions as f 
from pyspark.sql import Window
import sys
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.utils import getResolvedOptions
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType
# #inicio job
# args = getResolvedOptions(sys.argv, ['JOB_NAME'])
# sc = SparkContext()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)
# environment = 'der' 
# AWS_REGION = "us-east-1"

#config datas
dt_yesterday_referencia = datetime.datetime.now() - datetime.timedelta(days=1)
year_yesterday_referencia = dt_yesterday_referencia.year
month_yesterday_referencia = str(dt_yesterday_referencia.month).zfill(2)
day_yesterday_referencia= str(dt_yesterday_referencia.day).zfill(2)
date_yesterday_referencia = "%s%s%s" % (year_yesterday_referencia,  month_yesterday_referencia, day_yesterday_referencia)
date_yesterday_referencia_query = "%s%s%s%s%s%s" % (year_yesterday_referencia,"-",month_yesterday_referencia,"-",day_yesterday_referencia, '%')
date_today_processamento  = datetime.datetime.now() - datetime.timedelta(days=0)
year_today_processamento = date_today_processamento.year
month_today_processamento = str(date_today_processamento.month).zfill(2)
day_today_processamento = str(date_today_processamento.day).zfill(2)
date_today_processamento = "%s%s%s" % (year_today_processamento, month_today_processamento, day_today_processamento)


input_df= '%s%s%s'%('s3://pessoajuridicabucket-',environment,'/topics/mudar/mudar/')
output_df= '%s%s%s'%('s3://pessoajuridicabucket-',environment,'/bonus/boletos_emitidos/conversor/')

fonte_em_avro = glueContext.create_dynamic_frame_from_options(
    connection_type = 's3', 
    connection_options = 
    {'paths': 
    [input_df],\
    'recurse':True,\, 
    format="avro",\
    fonte_em_avro = "fonte_em_avro")


fonte = fonte_em_avro.toDF()
fonte.select('*').show(truncate=False)
fonte.printSchema()

fonte.select('data.texto_dado_beneficiario_cobranca.numero_agencia', \
             'data.texto_dado_beneficiario_cobranca.numero_conta_corrente',\
             'data.texto_dado_beneficiario_cobranca.numero_digito_verificador',\
             'data.cnpj',
             'data.valor_pago_boleto_cobranca')\


fonte.printSchema()
     
df = fonte\
    .withColumnRenamed("numero_conta_corrente","numero_conta_corrente_")\
    .withColumn('partition_agencia', fonte.numero_agencia.substr(1,1))
    .withColumnRenamed("numero_digito_verificador","numero_digito_verificador_") \
    .withColumnRenamed("numero_conta","numero_conta_corrente_") \
    .withColumnRenamed("numero_agencia", "numero_agencia_")\
    .withColumnRenamed("cnpj", "numero_cpf_cnpj_")


df.show()
df.printSchema()

df.printSchema()
#salvar para criar tabela,
BUCKET = 's3://dadosrelacionamento-'
BUCKET_OUTPUT_DF= '%s%s' % (BUCKET, '2')
output_sumarizador= '%s%s' % ('s3://pessoajuridicabucket','/bonus/')
from awsglue.dynamicframe import DynamicFrame
dif = DynamicFrame.fromDF(df, glueContext, 'test_nest')
datasink= glueContext.write_dynamic_frame.from_options(frame = dif,\
connection_type = 's3', \
connection_options = {'path': output_sumarizador,\
    'compression': 'snappy',\
      'partitionKeys': ['numero_digito_verificador_']},
    format='glueparquet',
transformation_ctx = 'datasink')

# # # job.commit()
# #;


In [ ]:
mock boleto liquidado

In [7]:
fonte_cadastral3 = glueContext.create_dynamic_frame_from_options(\
connection_type = "s3", 
connection_options = {"paths": ["s3://dadosrelacionamento-der/folder2/cadastral3.csv"]},\
format = "csv",format_options= {'withHeader': True, "separator":";"})



df3 = fonte_cadastral3.toDF()
df3.select('*').show(truncate=False)
df3.printSchema()

# #renomeia os campos para preparar eles para o join com outra tabela
df33 = df3\
        .withColumnRenamed("numero_conta_corrente","numero_conta_corrente_")\
        .withColumnRenamed("numero_digito_verificador","numero_digito_verificador_") \
        .withColumnRenamed("codigo_tipo_pessoa","codigo_tipo_pessoa_") \
        .withColumnRenamed("numero_conta","numero_conta_corrente_") \
        .withColumnRenamed("codigo_identificacao_cliente","codigo_identificacao_cliente_") \
        .withColumnRenamed("type","type_") \
        .withColumnRenamed("codigo_segmento", "codigo_segmento_")\
        .withColumnRenamed("numero_agencia", "numero_agencia_")

df33.show()

df.printSchema()
#salvar para criar tabela,
BUCKET = 's3://pessoajur'
BUCKET_OUTPUT_DF= '%s%s' % (BUCKET, '2')
output_sumarizador= '%s%s' % ('s3://pessoajur','/bonus/boletos_liquidados/conversor')
from awsglue.dynamicframe import DynamicFrame
dif3 = DynamicFrame.fromDF(df33, glueContext, 'test_nest')
datasink= glueContext.write_dynamic_frame.from_options(frame = dif3,\
connection_type = 's3', \
connection_options = {'path': output_sumarizador,\
    'compression': 'snappy',\
      'partitionKeys': ['numero_digito_verificador_']},
    format='glueparquet',
transformation_ctx = 'datasink')



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o250.getDynamicFrame.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: All access to this object has been disabled (Service: Amazon S3; Status Code: 403; Error Code: AllAccessDisabled; Request ID: VK01W47C0HW7N5KK; S3 Extended Request ID: 43+/HhxsXxR4Rb8VKD0E8GIdm243MYzqodu5LwvJ5JX0iqizmhXwiPs8ExXYRdrjcKiRxkTarU0=), S3 Extended Request ID: 43+/HhxsXxR4Rb8VKD0E8GIdm243MYzqodu5LwvJ5JX0iqizmhXwiPs8ExXYRdrjcKiRxkTarU0=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:286)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:781)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1440)
	at com.amazon.ws.emr.hadoop.fs.EmrFileSystem.exists(EmrFileSystem.java:347)
	at com.amazonaws.services.glue.util.FileSystemFolder.listFiles(FileLister.scala:227)
	at com.amazonaws.services.glu

In [ ]:
Boleto emitido
numero_agencia;numero_conta_corrente;numero_digito_verificador;valor_pago_boleto_cobranca;year;month;day;type;partition_agencia
111;12345;0;10;2022;5;23;BONUS_COBRANCA;1
111;12345;0;10;2022;5;23;BONUS_COBRANCA;1
555;12345;0;10;2022;5;23;BONUS_COBRANCA;1
555;12345;0;10;2022;5;23;BONUS_COBRANCA;1
5;40033;5;10;2022;5;23;BONUS_COBRANCA;4
3;40033;7;10;2022;5;23;BONUS_COBRANCA;4
3;40033;8;10;2022;5;23;BONUS_COBRANCA;4
5;22222;2;10;2022;5;23;BONUS_COBRANCA;2
6;22228;9;10;2022;5;23;BONUS_COBRANCA;3

In [ ]:
Cadastrais

In [ ]:
codigo_identificacao_cliente;codigo_segmento;codigo_tipo_pessoa;cnpj;numero_agencia_;numero_conta_corrente;numero_digito_verificador;type
00263ce9-5bc3-3c6a-9933-efe63d58d026;;1;859991720;119;40033;0;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d026;;1;859991720;3;40033;2;br.com.itau.depositos.conta_criada
20263ce9-5bc3-3c6a-9933-efe63d58d026;;2;2222222222;4;40033;8;br.com.itau.depositos.conta_criada
30263ce9-5bc3-3c6a-9933-efe63d58d026;;3;859991720;5;40033;7;br.com.itau.depositos.conta_criada
30263ce9-5bc3-3c6a-9933-efe63d58d026;;3;859991720;5;40033;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d026;;1;859991720;3;40033;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d026;;1;859991720;3;40033;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d027;;1;859991724;3;22222;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d028;;1;859991728;3;4411;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d029;;1;859991732;3;-13400;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d030;;1;859991736;3;-31211;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d031;;1;859991740;3;-49022;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d032;;1;859991744;3;-66833;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d033;;1;859991748;3;-84644;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d034;;1;859991752;3;-102455;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d035;;1;859991756;3;-120266;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d036;;1;859991760;3;-138077;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d037;;1;859991764;3;-155888;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d038;;1;859991768;3;-173699;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d039;;1;859991772;3;-191510;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d040;;1;859991776;3;-209321;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d041;;1;859991780;3;-227132;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d042;;1;859991784;3;-244943;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d043;;1;859991788;3;-262754;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d044;;1;859991792;3;-280565;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d045;;1;859991796;3;-298376;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d046;;1;859991800;3;-316187;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d047;;1;859991804;3;-333998;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d048;;1;859991808;3;-351809;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d049;;1;859991812;3;-369620;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d050;;1;859991816;3;-387431;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d051;;1;859991820;3;-405242;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d052;;1;859991824;3;-423053;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d053;;1;859991828;3;-440864;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d054;;1;859991832;3;-458675;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d055;;1;859991836;3;-476486;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d056;;1;859991840;3;-494297;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d057;;1;859991844;3;-512108;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d058;;1;859991848;3;-529919;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d059;;1;859991852;3;-547730;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d060;;1;859991856;3;-565541;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d061;;1;859991860;3;-583352;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d062;;1;859991864;3;-601163;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d063;;1;859991868;3;-618974;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d064;;1;859991872;3;-636785;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d065;;1;859991876;3;-654596;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d066;;1;859991880;3;-672407;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d067;;1;859991884;3;-690218;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d068;;1;859991888;3;-708029;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d069;;1;859991892;3;-725840;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d070;;1;859991896;3;-743651;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d071;;1;859991900;3;-761462;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d072;;1;859991904;3;-779273;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d073;;1;859991908;3;-797084;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d074;;1;859991912;3;-814895;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d075;;1;859991916;3;-832706;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d076;;1;859991920;3;-850517;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d077;;1;859991924;3;-868328;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d078;;1;859991928;3;-886139;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d079;;1;859991932;3;-903950;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d080;;1;859991936;3;-921761;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d081;;1;859991940;3;-939572;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d082;;1;859991944;3;-957383;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d083;;1;859991948;3;-975194;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d084;;1;859991952;3;-993005;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d085;;1;859991956;3;-1010816;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d086;;1;859991960;3;-1028627;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d087;;1;859991964;3;-1046438;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d088;;1;859991968;3;-1064249;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d089;;1;859991972;3;-1082060;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d090;;1;859991976;3;-1099871;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d091;;1;859991980;3;-1117682;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d092;;1;859991984;3;-1135493;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d093;;1;859991988;3;-1153304;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d094;;1;859991992;3;-1171115;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d095;;1;859991996;3;-1188926;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d096;;1;859992000;3;-1206737;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d097;;1;859992004;3;-1224548;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d098;;1;859992008;3;-1242359;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d099;;1;859992012;3;-1260170;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d100;;1;859992016;3;-1277981;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d101;;1;859992020;3;-1295792;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d102;;1;859992024;3;-1313603;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d103;;1;859992028;3;-1331414;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d104;;1;859992032;3;-1349225;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d105;;1;859992036;3;-1367036;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d106;;1;859992040;3;-1384847;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d107;;1;859992044;3;-1402658;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d108;;1;859992048;3;-1420469;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d109;;1;859992052;3;-1438280;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d110;;1;859992056;3;-1456091;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d111;;1;859992060;3;-1473902;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d112;;1;859992064;3;-1491713;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d113;;1;859992068;3;-1509524;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d114;;1;859992072;3;-1527335;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d115;;1;859992076;3;-1545146;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d116;;1;859992080;3;-1562957;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d117;;1;859992084;3;-1580768;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d118;;1;859992088;3;-1598579;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d119;;1;859992092;3;-1616390;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d120;;1;859992096;3;-1634201;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d121;;1;859992100;3;-1652012;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d122;;1;859992104;3;-1669823;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d123;;1;859992108;3;-1687634;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d124;;1;859992112;3;-1705445;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d125;;1;859992116;3;-1723256;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d126;;1;859992120;3;-1741067;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d127;;1;859992124;3;-1758878;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d128;;1;859992128;3;-1776689;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d129;;1;859992132;3;-1794500;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d130;;1;859992136;3;-1812311;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d131;;1;859992140;3;-1830122;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d132;;1;859992144;3;-1847933;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d133;;1;859992148;3;-1865744;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d134;;1;859992152;3;-1883555;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d135;;1;859992156;3;-1901366;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d136;;1;859992160;3;-1919177;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d137;;1;859992164;3;-1936988;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d138;;1;859992168;3;-1954799;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d139;;1;859992172;3;-1972610;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d140;;1;859992176;3;-1990421;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d141;;1;859992180;3;-2008232;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d142;;1;859992184;3;-2026043;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d143;;1;859992188;3;-2043854;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d144;;1;859992192;3;-2061665;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d145;;1;859992196;3;-2079476;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d146;;1;859992200;3;-2097287;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d147;;1;859992204;3;-2115098;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d148;;1;859992208;3;-2132909;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d149;;1;859992212;3;-2150720;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d150;;1;859992216;3;-2168531;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d151;;1;859992220;3;-2186342;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d152;;1;859992224;3;-2204153;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d153;;1;859992228;3;-2221964;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d154;;1;859992232;3;-2239775;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d155;;1;859992236;3;-2257586;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d156;;1;859992240;3;-2275397;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d157;;1;859992244;3;-2293208;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d158;;1;859992248;3;-2311019;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d159;;1;859992252;3;-2328830;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d160;;1;859992256;3;-2346641;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d161;;1;859992260;3;-2364452;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d162;;1;859992264;3;-2382263;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d163;;1;859992268;3;-2400074;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d164;;1;859992272;3;-2417885;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d165;;1;859992276;3;-2435696;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d166;;1;859992280;3;-2453507;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d167;;1;859992284;3;-2471318;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d168;;1;859992288;3;-2489129;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d169;;1;859992292;3;-2506940;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d170;;1;859992296;3;-2524751;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d171;;1;859992300;3;-2542562;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d172;;1;859992304;3;-2560373;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d173;;1;859992308;3;-2578184;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d174;;1;859992312;3;-2595995;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d175;;1;859992316;3;-2613806;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d176;;1;859992320;3;-2631617;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d177;;1;859992324;3;-2649428;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d178;;1;859992328;3;-2667239;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d179;;1;859992332;3;-2685050;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d180;;1;859992336;3;-2702861;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d181;;1;859992340;3;-2720672;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d182;;1;859992344;3;-2738483;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d183;;1;859992348;3;-2756294;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d184;;1;859992352;3;-2774105;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d185;;1;859992356;3;-2791916;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d186;;1;859992360;3;-2809727;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d187;;1;859992364;3;-2827538;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d188;;1;859992368;3;-2845349;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d189;;1;859992372;3;-2863160;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d190;;1;859992376;3;-2880971;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d191;;1;859992380;3;-2898782;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d192;;1;859992384;3;-2916593;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d193;;1;859992388;3;-2934404;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d194;;1;859992392;3;-2952215;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d195;;1;859992396;3;-2970026;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d196;;1;859992400;3;-2987837;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d197;;1;859992404;3;-3005648;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d198;;1;859992408;3;-3023459;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d199;;1;859992412;3;-3041270;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d200;;1;859992416;3;-3059081;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d201;;1;859992420;3;-3076892;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d202;;1;859992424;3;-3094703;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d203;;1;859992428;3;-3112514;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d204;;1;859992432;3;-3130325;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d205;;1;859992436;3;-3148136;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d206;;1;859992440;3;-3165947;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d207;;1;859992444;3;-3183758;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d208;;1;859992448;3;-3201569;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d209;;1;859992452;3;-3219380;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d210;;1;859992456;3;-3237191;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d211;;1;859992460;3;-3255002;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d212;;1;859992464;3;-3272813;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d213;;1;859992468;3;-3290624;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d214;;1;859992472;3;-3308435;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d215;;1;859992476;3;-3326246;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d216;;1;859992480;3;-3344057;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d217;;1;859992484;3;-3361868;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d218;;1;859992488;3;-3379679;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d219;;1;859992492;3;-3397490;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d220;;1;859992496;3;-3415301;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d221;;1;859992500;3;-3433112;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d222;;1;859992504;3;-3450923;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d223;;1;859992508;3;-3468734;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d224;;1;859992512;3;-3486545;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d225;;1;859992516;3;-3504356;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d226;;1;859992520;3;-3522167;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d227;;1;859992524;3;-3539978;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d228;;1;859992528;3;-3557789;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d229;;1;859992532;3;-3575600;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d230;;1;859992536;3;-3593411;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d231;;1;859992540;3;-3611222;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d232;;1;859992544;3;-3629033;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d233;;1;859992548;3;-3646844;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d234;;1;859992552;3;-3664655;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d235;;1;859992556;3;-3682466;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d236;;1;859992560;3;-3700277;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d237;;1;859992564;3;-3718088;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d238;;1;859992568;3;-3735899;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d239;;1;859992572;3;-3753710;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d240;;1;859992576;3;-3771521;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d241;;1;859992580;3;-3789332;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d242;;1;859992584;3;-3807143;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d243;;1;859992588;3;-3824954;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d244;;1;859992592;3;-3842765;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d245;;1;859992596;3;-3860576;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d246;;1;859992600;3;-3878387;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d247;;1;859992604;3;-3896198;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d248;;1;859992608;3;-3914009;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d249;;1;859992612;3;-3931820;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d250;;1;859992616;3;-3949631;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d251;;1;859992620;3;-3967442;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d252;;1;859992624;3;-3985253;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d253;;1;859992628;3;-4003064;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d254;;1;859992632;3;-4020875;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d255;;1;859992636;3;-4038686;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d256;;1;859992640;3;-4056497;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d257;;1;859992644;3;-4074308;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d258;;1;859992648;3;-4092119;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d259;;1;859992652;3;-4109930;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d260;;1;859992656;3;-4127741;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d261;;1;859992660;3;-4145552;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d262;;1;859992664;3;-4163363;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d263;;1;859992668;3;-4181174;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d264;;1;859992672;3;-4198985;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d265;;1;859992676;3;-4216796;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d266;;1;859992680;3;-4234607;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d267;;1;859992684;3;-4252418;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d268;;1;859992688;3;-4270229;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d269;;1;859992692;3;-4288040;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d270;;1;859992696;3;-4305851;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d271;;1;859992700;3;-4323662;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d272;;1;859992704;3;-4341473;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d273;;1;859992708;3;-4359284;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d274;;1;859992712;3;-4377095;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d275;;1;859992716;3;-4394906;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d276;;1;859992720;3;-4412717;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d277;;1;859992724;3;-4430528;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d278;;1;859992728;3;-4448339;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d279;;1;859992732;3;-4466150;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d280;;1;859992736;3;-4483961;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d281;;1;859992740;3;-4501772;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d282;;1;859992744;3;-4519583;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d283;;1;859992748;3;-4537394;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d284;;1;859992752;3;-4555205;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d285;;1;859992756;3;-4573016;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d286;;1;859992760;3;-4590827;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d287;;1;859992764;3;-4608638;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d288;;1;859992768;3;-4626449;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d289;;1;859992772;3;-4644260;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d290;;1;859992776;3;-4662071;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d291;;1;859992780;3;-4679882;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d292;;1;859992784;3;-4697693;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d293;;1;859992788;3;-4715504;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d294;;1;859992792;3;-4733315;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d295;;1;859992796;3;-4751126;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d296;;1;859992800;3;-4768937;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d297;;1;859992804;3;-4786748;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d298;;1;859992808;3;-4804559;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d299;;1;859992812;3;-4822370;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d300;;1;859992816;3;-4840181;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d301;;1;859992820;3;-4857992;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d302;;1;859992824;3;-4875803;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d303;;1;859992828;3;-4893614;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d304;;1;859992832;3;-4911425;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d305;;1;859992836;3;-4929236;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d306;;1;859992840;3;-4947047;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d307;;1;859992844;3;-4964858;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d308;;1;859992848;3;-4982669;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d309;;1;859992852;3;-5000480;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d310;;1;859992856;3;-5018291;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d311;;1;859992860;3;-5036102;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d312;;1;859992864;3;-5053913;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d313;;1;859992868;3;-5071724;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d314;;1;859992872;3;-5089535;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d315;;1;859992876;3;-5107346;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d316;;1;859992880;3;-5125157;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d317;;1;859992884;3;-5142968;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d318;;1;859992888;3;-5160779;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d319;;1;859992892;3;-5178590;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d320;;1;859992896;3;-5196401;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d321;;1;859992900;3;-5214212;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d322;;1;859992904;3;-5232023;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d323;;1;859992908;3;-5249834;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d324;;1;859992912;3;-5267645;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d325;;1;859992916;3;-5285456;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d326;;1;859992920;3;-5303267;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d327;;1;859992924;3;-5321078;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d328;;1;859992928;3;-5338889;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d329;;1;859992932;3;-5356700;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d330;;1;859992936;3;-5374511;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d331;;1;859992940;3;-5392322;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d332;;1;859992944;3;-5410133;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d333;;1;859992948;3;-5427944;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d334;;1;859992952;3;-5445755;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d335;;1;859992956;3;-5463566;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d336;;1;859992960;3;-5481377;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d337;;1;859992964;3;-5499188;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d338;;1;859992968;3;-5516999;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d339;;1;859992972;3;-5534810;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d340;;1;859992976;3;-5552621;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d341;;1;859992980;3;-5570432;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d342;;1;859992984;3;-5588243;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d343;;1;859992988;3;-5606054;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d344;;1;859992992;3;-5623865;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d345;;1;859992996;3;-5641676;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d346;;1;859993000;3;-5659487;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d347;;1;859993004;3;-5677298;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d348;;1;859993008;3;-5695109;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d349;;1;859993012;3;-5712920;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d350;;1;859993016;3;-5730731;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d351;;1;859993020;3;-5748542;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d352;;1;859993024;3;-5766353;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d353;;1;859993028;3;-5784164;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d354;;1;859993032;3;-5801975;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d355;;1;859993036;3;-5819786;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d356;;1;859993040;3;-5837597;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d357;;1;859993044;3;-5855408;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d358;;1;859993048;3;-5873219;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d359;;1;859993052;3;-5891030;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d360;;1;859993056;3;-5908841;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d361;;1;859993060;3;-5926652;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d362;;1;859993064;3;-5944463;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d363;;1;859993068;3;-5962274;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d364;;1;859993072;3;-5980085;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d365;;1;859993076;3;-5997896;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d366;;1;859993080;3;-6015707;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d367;;1;859993084;3;-6033518;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d368;;1;859993088;3;-6051329;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d369;;1;859993092;3;-6069140;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d370;;1;859993096;3;-6086951;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d371;;1;859993100;3;-6104762;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d372;;1;859993104;3;-6122573;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d373;;1;859993108;3;-6140384;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d374;;1;859993112;3;-6158195;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d375;;1;859993116;3;-6176006;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d376;;1;859993120;3;-6193817;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d377;;1;859993124;3;-6211628;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d378;;1;859993128;3;-6229439;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d379;;1;859993132;3;-6247250;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d380;;1;859993136;3;-6265061;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d381;;1;859993140;3;-6282872;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d382;;1;859993144;3;-6300683;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d383;;1;859993148;3;-6318494;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d384;;1;859993152;3;-6336305;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d385;;1;859993156;3;-6354116;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d386;;1;859993160;3;-6371927;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d387;;1;859993164;3;-6389738;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d388;;1;859993168;3;-6407549;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d389;;1;859993172;3;-6425360;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d390;;1;859993176;3;-6443171;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d391;;1;859993180;3;-6460982;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d392;;1;859993184;3;-6478793;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d393;;1;859993188;3;-6496604;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d394;;1;859993192;3;-6514415;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d395;;1;859993196;3;-6532226;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d396;;1;859993200;3;-6550037;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d397;;1;859993204;3;-6567848;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d398;;1;859993208;3;-6585659;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d399;;1;859993212;3;-6603470;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d400;;1;859993216;3;-6621281;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d401;;1;859993220;3;-6639092;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d402;;1;859993224;3;-6656903;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d403;;1;859993228;3;-6674714;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d404;;1;859993232;3;-6692525;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d405;;1;859993236;3;-6710336;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d406;;1;859993240;3;-6728147;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d407;;1;859993244;3;-6745958;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d408;;1;859993248;3;-6763769;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d409;;1;859993252;3;-6781580;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d410;;1;859993256;3;-6799391;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d411;;1;859993260;3;-6817202;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d412;;1;859993264;3;-6835013;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d413;;1;859993268;3;-6852824;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d414;;1;859993272;3;-6870635;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d415;;1;859993276;3;-6888446;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d416;;1;859993280;3;-6906257;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d417;;1;859993284;3;-6924068;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d418;;1;859993288;3;-6941879;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d419;;1;859993292;3;-6959690;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d420;;1;859993296;3;-6977501;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d421;;1;859993300;3;-6995312;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d422;;1;859993304;3;-7013123;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d423;;1;859993308;3;-7030934;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d424;;1;859993312;3;-7048745;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d425;;1;859993316;3;-7066556;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d426;;1;859993320;3;-7084367;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d427;;1;859993324;3;-7102178;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d428;;1;859993328;3;-7119989;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d429;;1;859993332;3;-7137800;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d430;;1;859993336;3;-7155611;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d431;;1;859993340;3;-7173422;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d432;;1;859993344;3;-7191233;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d433;;1;859993348;3;-7209044;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d434;;1;859993352;3;-7226855;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d435;;1;859993356;3;-7244666;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d436;;1;859993360;3;-7262477;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d437;;1;859993364;3;-7280288;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d438;;1;859993368;3;-7298099;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d439;;1;859993372;3;-7315910;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d440;;1;859993376;3;-7333721;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d441;;1;859993380;3;-7351532;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d442;;1;859993384;3;-7369343;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d443;;1;859993388;3;-7387154;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d444;;1;859993392;3;-7404965;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d445;;1;859993396;3;-7422776;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d446;;1;859993400;3;-7440587;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d447;;1;859993404;3;-7458398;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d448;;1;859993408;3;-7476209;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d449;;1;859993412;3;-7494020;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d450;;1;859993416;3;-7511831;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d451;;1;859993420;3;-7529642;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d452;;1;859993424;3;-7547453;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d453;;1;859993428;3;-7565264;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d454;;1;859993432;3;-7583075;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d455;;1;859993436;3;-7600886;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d456;;1;859993440;3;-7618697;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d457;;1;859993444;3;-7636508;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d458;;1;859993448;3;-7654319;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d459;;1;859993452;3;-7672130;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d460;;1;859993456;3;-7689941;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d461;;1;859993460;3;-7707752;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d462;;1;859993464;3;-7725563;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d463;;1;859993468;3;-7743374;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d464;;1;859993472;3;-7761185;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d465;;1;859993476;3;-7778996;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d466;;1;859993480;3;-7796807;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d467;;1;859993484;3;-7814618;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d468;;1;859993488;3;-7832429;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d469;;1;859993492;3;-7850240;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d470;;1;859993496;3;-7868051;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d471;;1;859993500;3;-7885862;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d472;;1;859993504;3;-7903673;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d473;;1;859993508;3;-7921484;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d474;;1;859993512;3;-7939295;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d475;;1;859993516;3;-7957106;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d476;;1;859993520;3;-7974917;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d477;;1;859993524;3;-7992728;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d478;;1;859993528;3;-8010539;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d479;;1;859993532;3;-8028350;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d480;;1;859993536;3;-8046161;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d481;;1;859993540;3;-8063972;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d482;;1;859993544;3;-8081783;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d483;;1;859993548;3;-8099594;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d484;;1;859993552;3;-8117405;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d485;;1;859993556;3;-8135216;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d486;;1;859993560;3;-8153027;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d487;;1;859993564;3;-8170838;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d488;;1;859993568;3;-8188649;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d489;;1;859993572;3;-8206460;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d490;;1;859993576;3;-8224271;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d491;;1;859993580;3;-8242082;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d492;;1;859993584;3;-8259893;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d493;;1;859993588;3;-8277704;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d494;;1;859993592;3;-8295515;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d495;;1;859993596;3;-8313326;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d496;;1;859993600;3;-8331137;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d497;;1;859993604;3;-8348948;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d498;;1;859993608;3;-8366759;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d499;;1;859993612;3;-8384570;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d500;;1;859993616;3;-8402381;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d501;;1;859993620;3;-8420192;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d502;;1;859993624;3;-8438003;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d503;;1;859993628;3;-8455814;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d504;;1;859993632;3;-8473625;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d505;;1;859993636;3;-8491436;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d506;;1;859993640;3;-8509247;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d507;;1;859993644;3;-8527058;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d508;;1;859993648;3;-8544869;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d509;;1;859993652;3;-8562680;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d510;;1;859993656;3;-8580491;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d511;;1;859993660;3;-8598302;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d512;;1;859993664;3;-8616113;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d513;;1;859993668;3;-8633924;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d514;;1;859993672;3;-8651735;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d515;;1;859993676;3;-8669546;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d516;;1;859993680;3;-8687357;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d517;;1;859993684;3;-8705168;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d518;;1;859993688;3;-8722979;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d519;;1;859993692;3;-8740790;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d520;;1;859993696;3;-8758601;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d521;;1;859993700;3;-8776412;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d522;;1;859993704;3;-8794223;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d523;;1;859993708;3;-8812034;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d524;;1;859993712;3;-8829845;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d525;;1;859993716;3;-8847656;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d526;;1;859993720;3;-8865467;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d527;;1;859993724;3;-8883278;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d528;;1;859993728;3;-8901089;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d529;;1;859993732;3;-8918900;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d530;;1;859993736;3;-8936711;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d531;;1;859993740;3;-8954522;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d532;;1;859993744;3;-8972333;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d533;;1;859993748;3;-8990144;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d534;;1;859993752;3;-9007955;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d535;;1;859993756;3;-9025766;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d536;;1;859993760;3;-9043577;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d537;;1;859993764;3;-9061388;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d538;;1;859993768;3;-9079199;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d539;;1;859993772;3;-9097010;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d540;;1;859993776;3;-9114821;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d541;;1;859993780;3;-9132632;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d542;;1;859993784;3;-9150443;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d543;;1;859993788;3;-9168254;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d544;;1;859993792;3;-9186065;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d545;;1;859993796;3;-9203876;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d546;;1;859993800;3;-9221687;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d547;;1;859993804;3;-9239498;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d548;;1;859993808;3;-9257309;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d549;;1;859993812;3;-9275120;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d550;;1;859993816;3;-9292931;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d551;;1;859993820;3;-9310742;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d552;;1;859993824;3;-9328553;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d553;;1;859993828;3;-9346364;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d554;;1;859993832;3;-9364175;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d555;;1;859993836;3;-9381986;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d556;;1;859993840;3;-9399797;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d557;;1;859993844;3;-9417608;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d558;;1;859993848;3;-9435419;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d559;;1;859993852;3;-9453230;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d560;;1;859993856;3;-9471041;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d561;;1;859993860;3;-9488852;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d562;;1;859993864;3;-9506663;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d563;;1;859993868;3;-9524474;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d564;;1;859993872;3;-9542285;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d565;;1;859993876;3;-9560096;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d566;;1;859993880;3;-9577907;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d567;;1;859993884;3;-9595718;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d568;;1;859993888;3;-9613529;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d569;;1;859993892;3;-9631340;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d570;;1;859993896;3;-9649151;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d571;;1;859993900;3;-9666962;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d572;;1;859993904;3;-9684773;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d573;;1;859993908;3;-9702584;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d574;;1;859993912;3;-9720395;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d575;;1;859993916;3;-9738206;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d576;;1;859993920;3;-9756017;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d577;;1;859993924;3;-9773828;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d578;;1;859993928;3;-9791639;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d579;;1;859993932;3;-9809450;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d580;;1;859993936;3;-9827261;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d581;;1;859993940;3;-9845072;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d582;;1;859993944;3;-9862883;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d583;;1;859993948;3;-9880694;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d584;;1;859993952;3;-9898505;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d585;;1;859993956;3;-9916316;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d586;;1;859993960;3;-9934127;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d587;;1;859993964;3;-9951938;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d588;;1;859993968;3;-9969749;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d589;;1;859993972;3;-9987560;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d590;;1;859993976;3;-10005371;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d591;;1;859993980;3;-10023182;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d592;;1;859993984;3;-10040993;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d593;;1;859993988;3;-10058804;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d594;;1;859993992;3;-10076615;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d595;;1;859993996;3;-10094426;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d596;;1;859994000;3;-10112237;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d597;;1;859994004;3;-10130048;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d598;;1;859994008;3;-10147859;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d599;;1;859994012;3;-10165670;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d600;;1;859994016;3;-10183481;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d601;;1;859994020;3;-10201292;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d602;;1;859994024;3;-10219103;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d603;;1;859994028;3;-10236914;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d604;;1;859994032;3;-10254725;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d605;;1;859994036;3;-10272536;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d606;;1;859994040;3;-10290347;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d607;;1;859994044;3;-10308158;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d608;;1;859994048;3;-10325969;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d609;;1;859994052;3;-10343780;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d610;;1;859994056;3;-10361591;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d611;;1;859994060;3;-10379402;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d612;;1;859994064;3;-10397213;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d613;;1;859994068;3;-10415024;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d614;;1;859994072;3;-10432835;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d615;;1;859994076;3;-10450646;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d616;;1;859994080;3;-10468457;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d617;;1;859994084;3;-10486268;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d618;;1;859994088;3;-10504079;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d619;;1;859994092;3;-10521890;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d620;;1;859994096;3;-10539701;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d621;;1;859994100;3;-10557512;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d622;;1;859994104;3;-10575323;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d623;;1;859994108;3;-10593134;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d624;;1;859994112;3;-10610945;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d625;;1;859994116;3;-10628756;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d626;;1;859994120;3;-10646567;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d627;;1;859994124;3;-10664378;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d628;;1;859994128;3;-10682189;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d629;;1;859994132;3;-10700000;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d630;;1;859994136;3;-10717811;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d631;;1;859994140;3;-10735622;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d632;;1;859994144;3;-10753433;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d633;;1;859994148;3;-10771244;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d634;;1;859994152;3;-10789055;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d635;;1;859994156;3;-10806866;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d636;;1;859994160;3;-10824677;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d637;;1;859994164;3;-10842488;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d638;;1;859994168;3;-10860299;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d639;;1;859994172;3;-10878110;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d640;;1;859994176;3;-10895921;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d641;;1;859994180;3;-10913732;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d642;;1;859994184;3;-10931543;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d643;;1;859994188;3;-10949354;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d644;;1;859994192;3;-10967165;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d645;;1;859994196;3;-10984976;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d646;;1;859994200;3;-11002787;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d647;;1;859994204;3;-11020598;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d648;;1;859994208;3;-11038409;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d649;;1;859994212;3;-11056220;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d650;;1;859994216;3;-11074031;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d651;;1;859994220;3;-11091842;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d652;;1;859994224;3;-11109653;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d653;;1;859994228;3;-11127464;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d654;;1;859994232;3;-11145275;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d655;;1;859994236;3;-11163086;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d656;;1;859994240;3;-11180897;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d657;;1;859994244;3;-11198708;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d658;;1;859994248;3;-11216519;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d659;;1;859994252;3;-11234330;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d660;;1;859994256;3;-11252141;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d661;;1;859994260;3;-11269952;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d662;;1;859994264;3;-11287763;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d663;;1;859994268;3;-11305574;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d664;;1;859994272;3;-11323385;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d665;;1;859994276;3;-11341196;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d666;;1;859994280;3;-11359007;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d667;;1;859994284;3;-11376818;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d668;;1;859994288;3;-11394629;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d669;;1;859994292;3;-11412440;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d670;;1;859994296;3;-11430251;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d671;;1;859994300;3;-11448062;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d672;;1;859994304;3;-11465873;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d673;;1;859994308;3;-11483684;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d674;;1;859994312;3;-11501495;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d675;;1;859994316;3;-11519306;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d676;;1;859994320;3;-11537117;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d677;;1;859994324;3;-11554928;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d678;;1;859994328;3;-11572739;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d679;;1;859994332;3;-11590550;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d680;;1;859994336;3;-11608361;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d681;;1;859994340;3;-11626172;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d682;;1;859994344;3;-11643983;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d683;;1;859994348;3;-11661794;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d684;;1;859994352;3;-11679605;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d685;;1;859994356;3;-11697416;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d686;;1;859994360;3;-11715227;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d687;;1;859994364;3;-11733038;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d688;;1;859994368;3;-11750849;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d689;;1;859994372;3;-11768660;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d690;;1;859994376;3;-11786471;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d691;;1;859994380;3;-11804282;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d692;;1;859994384;3;-11822093;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d693;;1;859994388;3;-11839904;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d694;;1;859994392;3;-11857715;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d695;;1;859994396;3;-11875526;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d696;;1;859994400;3;-11893337;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d697;;1;859994404;3;-11911148;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d698;;1;859994408;3;-11928959;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d699;;1;859994412;3;-11946770;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d700;;1;859994416;3;-11964581;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d701;;1;859994420;3;-11982392;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d702;;1;859994424;3;-12000203;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d703;;1;859994428;3;-12018014;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d704;;1;859994432;3;-12035825;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d705;;1;859994436;3;-12053636;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d706;;1;859994440;3;-12071447;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d707;;1;859994444;3;-12089258;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d708;;1;859994448;3;-12107069;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d709;;1;859994452;3;-12124880;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d710;;1;859994456;3;-12142691;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d711;;1;859994460;3;-12160502;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d712;;1;859994464;3;-12178313;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d713;;1;859994468;3;-12196124;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d714;;1;859994472;3;-12213935;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d715;;1;859994476;3;-12231746;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d716;;1;859994480;3;-12249557;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d717;;1;859994484;3;-12267368;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d718;;1;859994488;3;-12285179;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d719;;1;859994492;3;-12302990;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d720;;1;859994496;3;-12320801;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d721;;1;859994500;3;-12338612;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d722;;1;859994504;3;-12356423;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d723;;1;859994508;3;-12374234;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d724;;1;859994512;3;-12392045;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d725;;1;859994516;3;-12409856;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d726;;1;859994520;3;-12427667;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d727;;1;859994524;3;-12445478;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d728;;1;859994528;3;-12463289;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d729;;1;859994532;3;-12481100;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d730;;1;859994536;3;-12498911;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d731;;1;859994540;3;-12516722;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d732;;1;859994544;3;-12534533;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d733;;1;859994548;3;-12552344;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d734;;1;859994552;3;-12570155;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d735;;1;859994556;3;-12587966;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d736;;1;859994560;3;-12605777;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d737;;1;859994564;3;-12623588;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d738;;1;859994568;3;-12641399;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d739;;1;859994572;3;-12659210;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d740;;1;859994576;3;-12677021;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d741;;1;859994580;3;-12694832;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d742;;1;859994584;3;-12712643;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d743;;1;859994588;3;-12730454;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d744;;1;859994592;3;-12748265;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d745;;1;859994596;3;-12766076;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d746;;1;859994600;3;-12783887;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d747;;1;859994604;3;-12801698;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d748;;1;859994608;3;-12819509;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d749;;1;859994612;3;-12837320;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d750;;1;859994616;3;-12855131;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d751;;1;859994620;3;-12872942;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d752;;1;859994624;3;-12890753;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d753;;1;859994628;3;-12908564;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d754;;1;859994632;3;-12926375;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d755;;1;859994636;3;-12944186;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d756;;1;859994640;3;-12961997;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d757;;1;859994644;3;-12979808;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d758;;1;859994648;3;-12997619;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d759;;1;859994652;3;-13015430;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d760;;1;859994656;3;-13033241;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d761;;1;859994660;3;-13051052;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d762;;1;859994664;3;-13068863;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d763;;1;859994668;3;-13086674;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d764;;1;859994672;3;-13104485;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d765;;1;859994676;3;-13122296;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d766;;1;859994680;3;-13140107;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d767;;1;859994684;3;-13157918;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d768;;1;859994688;3;-13175729;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d769;;1;859994692;3;-13193540;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d770;;1;859994696;3;-13211351;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d771;;1;859994700;3;-13229162;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d772;;1;859994704;3;-13246973;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d773;;1;859994708;3;-13264784;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d774;;1;859994712;3;-13282595;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d775;;1;859994716;3;-13300406;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d776;;1;859994720;3;-13318217;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d777;;1;859994724;3;-13336028;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d778;;1;859994728;3;-13353839;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d779;;1;859994732;3;-13371650;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d780;;1;859994736;3;-13389461;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d781;;1;859994740;3;-13407272;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d782;;1;859994744;3;-13425083;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d783;;1;859994748;3;-13442894;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d784;;1;859994752;3;-13460705;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d785;;1;859994756;3;-13478516;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d786;;1;859994760;3;-13496327;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d787;;1;859994764;3;-13514138;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d788;;1;859994768;3;-13531949;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d789;;1;859994772;3;-13549760;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d790;;1;859994776;3;-13567571;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d791;;1;859994780;3;-13585382;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d792;;1;859994784;3;-13603193;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d793;;1;859994788;3;-13621004;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d794;;1;859994792;3;-13638815;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d795;;1;859994796;3;-13656626;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d796;;1;859994800;3;-13674437;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d797;;1;859994804;3;-13692248;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d798;;1;859994808;3;-13710059;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d799;;1;859994812;3;-13727870;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d800;;1;859994816;3;-13745681;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d801;;1;859994820;3;-13763492;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d802;;1;859994824;3;-13781303;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d803;;1;859994828;3;-13799114;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d804;;1;859994832;3;-13816925;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d805;;1;859994836;3;-13834736;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d806;;1;859994840;3;-13852547;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d807;;1;859994844;3;-13870358;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d808;;1;859994848;3;-13888169;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d809;;1;859994852;3;-13905980;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d810;;1;859994856;3;-13923791;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d811;;1;859994860;3;-13941602;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d812;;1;859994864;3;-13959413;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d813;;1;859994868;3;-13977224;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d814;;1;859994872;3;-13995035;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d815;;1;859994876;3;-14012846;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d816;;1;859994880;3;-14030657;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d817;;1;859994884;3;-14048468;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d818;;1;859994888;3;-14066279;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d819;;1;859994892;3;-14084090;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d820;;1;859994896;3;-14101901;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d821;;1;859994900;3;-14119712;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d822;;1;859994904;3;-14137523;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d823;;1;859994908;3;-14155334;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d824;;1;859994912;3;-14173145;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d825;;1;859994916;3;-14190956;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d826;;1;859994920;3;-14208767;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d827;;1;859994924;3;-14226578;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d828;;1;859994928;3;-14244389;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d829;;1;859994932;3;-14262200;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d830;;1;859994936;3;-14280011;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d831;;1;859994940;3;-14297822;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d832;;1;859994944;3;-14315633;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d833;;1;859994948;3;-14333444;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d834;;1;859994952;3;-14351255;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d835;;1;859994956;3;-14369066;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d836;;1;859994960;3;-14386877;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d837;;1;859994964;3;-14404688;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d838;;1;859994968;3;-14422499;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d839;;1;859994972;3;-14440310;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d840;;1;859994976;3;-14458121;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d841;;1;859994980;3;-14475932;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d842;;1;859994984;3;-14493743;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d843;;1;859994988;3;-14511554;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d844;;1;859994992;3;-14529365;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d845;;1;859994996;3;-14547176;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d846;;1;859995000;3;-14564987;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d847;;1;859995004;3;-14582798;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d848;;1;859995008;3;-14600609;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d849;;1;859995012;3;-14618420;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d850;;1;859995016;3;-14636231;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d851;;1;859995020;3;-14654042;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d852;;1;859995024;3;-14671853;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d853;;1;859995028;3;-14689664;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d854;;1;859995032;3;-14707475;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d855;;1;859995036;3;-14725286;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d856;;1;859995040;3;-14743097;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d857;;1;859995044;3;-14760908;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d858;;1;859995048;3;-14778719;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d859;;1;859995052;3;-14796530;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d860;;1;859995056;3;-14814341;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d861;;1;859995060;3;-14832152;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d862;;1;859995064;3;-14849963;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d863;;1;859995068;3;-14867774;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d864;;1;859995072;3;-14885585;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d865;;1;859995076;3;-14903396;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d866;;1;859995080;3;-14921207;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d867;;1;859995084;3;-14939018;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d868;;1;859995088;3;-14956829;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d869;;1;859995092;3;-14974640;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d870;;1;859995096;3;-14992451;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d871;;1;859995100;3;-15010262;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d872;;1;859995104;3;-15028073;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d873;;1;859995108;3;-15045884;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d874;;1;859995112;3;-15063695;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d875;;1;859995116;3;-15081506;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d876;;1;859995120;3;-15099317;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d877;;1;859995124;3;-15117128;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d878;;1;859995128;3;-15134939;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d879;;1;859995132;3;-15152750;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d880;;1;859995136;3;-15170561;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d881;;1;859995140;3;-15188372;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d882;;1;859995144;3;-15206183;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d883;;1;859995148;3;-15223994;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d884;;1;859995152;3;-15241805;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d885;;1;859995156;3;-15259616;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d886;;1;859995160;3;-15277427;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d887;;1;859995164;3;-15295238;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d888;;1;859995168;3;-15313049;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d889;;1;859995172;3;-15330860;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d890;;1;859995176;3;-15348671;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d891;;1;859995180;3;-15366482;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d892;;1;859995184;3;-15384293;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d893;;1;859995188;3;-15402104;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d894;;1;859995192;3;-15419915;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d895;;1;859995196;3;-15437726;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d896;;1;859995200;3;-15455537;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d897;;1;859995204;3;-15473348;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d898;;1;859995208;3;-15491159;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d899;;1;859995212;3;-15508970;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d900;;1;859995216;3;-15526781;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d901;;1;859995220;3;-15544592;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d902;;1;859995224;3;-15562403;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d903;;1;859995228;3;-15580214;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d904;;1;859995232;3;-15598025;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d905;;1;859995236;3;-15615836;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d906;;1;859995240;3;-15633647;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d907;;1;859995244;3;-15651458;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d908;;1;859995248;3;-15669269;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d909;;1;859995252;3;-15687080;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d910;;1;859995256;3;-15704891;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d911;;1;859995260;3;-15722702;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d912;;1;859995264;3;-15740513;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d913;;1;859995268;3;-15758324;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d914;;1;859995272;3;-15776135;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d915;;1;859995276;3;-15793946;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d916;;1;859995280;3;-15811757;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d917;;1;859995284;3;-15829568;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d918;;1;859995288;3;-15847379;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d919;;1;859995292;3;-15865190;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d920;;1;859995296;3;-15883001;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d921;;1;859995300;3;-15900812;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d922;;1;859995304;3;-15918623;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d923;;1;859995308;3;-15936434;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d924;;1;859995312;3;-15954245;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d925;;1;859995316;3;-15972056;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d926;;1;859995320;3;-15989867;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d927;;1;859995324;3;-16007678;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d928;;1;859995328;3;-16025489;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d929;;1;859995332;3;-16043300;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d930;;1;859995336;3;-16061111;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d931;;1;859995340;3;-16078922;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d932;;1;859995344;3;-16096733;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d933;;1;859995348;3;-16114544;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d934;;1;859995352;3;-16132355;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d935;;1;859995356;3;-16150166;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d936;;1;859995360;3;-16167977;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d937;;1;859995364;3;-16185788;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d938;;1;859995368;3;-16203599;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d939;;1;859995372;3;-16221410;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d940;;1;859995376;3;-16239221;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d941;;1;859995380;3;-16257032;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d942;;1;859995384;3;-16274843;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d943;;1;859995388;3;-16292654;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d944;;1;859995392;3;-16310465;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d945;;1;859995396;3;-16328276;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d946;;1;859995400;3;-16346087;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d947;;1;859995404;3;-16363898;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d948;;1;859995408;3;-16381709;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d949;;1;859995412;3;-16399520;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d950;;1;859995416;3;-16417331;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d951;;1;859995420;3;-16435142;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d952;;1;859995424;3;-16452953;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d953;;1;859995428;3;-16470764;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d954;;1;859995432;3;-16488575;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d955;;1;859995436;3;-16506386;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d956;;1;859995440;3;-16524197;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d957;;1;859995444;3;-16542008;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d958;;1;859995448;3;-16559819;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d959;;1;859995452;3;-16577630;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d960;;1;859995456;3;-16595441;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d961;;1;859995460;3;-16613252;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d962;;1;859995464;3;-16631063;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d963;;1;859995468;3;-16648874;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d964;;1;859995472;3;-16666685;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d965;;1;859995476;3;-16684496;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d966;;1;859995480;3;-16702307;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d967;;1;859995484;3;-16720118;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d968;;1;859995488;3;-16737929;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d969;;1;859995492;3;-16755740;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d970;;1;859995496;3;-16773551;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d971;;1;859995500;3;-16791362;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d972;;1;859995504;3;-16809173;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d973;;1;859995508;3;-16826984;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d974;;1;859995512;3;-16844795;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d975;;1;859995516;3;-16862606;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d976;;1;859995520;3;-16880417;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d977;;1;859995524;3;-16898228;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d978;;1;859995528;3;-16916039;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d979;;1;859995532;3;-16933850;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d980;;1;859995536;3;-16951661;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d981;;1;859995540;3;-16969472;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d982;;1;859995544;3;-16987283;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d983;;1;859995548;3;-17005094;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d984;;1;859995552;3;-17022905;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d985;;1;859995556;3;-17040716;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d986;;1;859995560;3;-17058527;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d987;;1;859995564;3;-17076338;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d988;;1;859995568;3;-17094149;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d989;;1;859995572;3;-17111960;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d990;;1;859995576;3;-17129771;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d991;;1;859995580;3;-17147582;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d992;;1;859995584;3;-17165393;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d993;;1;859995588;3;-17183204;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d994;;1;859995592;3;-17201015;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d995;;1;859995596;3;-17218826;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d996;;1;859995600;3;-17236637;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d997;;1;859995604;3;-17254448;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d998;;1;859995608;3;-17272259;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d999;;1;859995612;3;-17290070;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1000;;1;859995616;3;-17307881;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1001;;1;859995620;3;-17325692;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1002;;1;859995624;3;-17343503;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1003;;1;859995628;3;-17361314;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1004;;1;859995632;3;-17379125;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1005;;1;859995636;3;-17396936;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1006;;1;859995640;3;-17414747;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1007;;1;859995644;3;-17432558;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1008;;1;859995648;3;-17450369;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1009;;1;859995652;3;-17468180;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1010;;1;859995656;3;-17485991;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1011;;1;859995660;3;-17503802;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1012;;1;859995664;3;-17521613;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1013;;1;859995668;3;-17539424;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1014;;1;859995672;3;-17557235;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1015;;1;859995676;3;-17575046;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1016;;1;859995680;3;-17592857;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1017;;1;859995684;3;-17610668;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1018;;1;859995688;3;-17628479;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1019;;1;859995692;3;-17646290;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1020;;1;859995696;3;-17664101;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1021;;1;859995700;3;-17681912;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1022;;1;859995704;3;-17699723;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1023;;1;859995708;3;-17717534;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1024;;1;859995712;3;-17735345;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1025;;1;859995716;3;-17753156;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1026;;1;859995720;3;-17770967;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1027;;1;859995724;3;-17788778;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1028;;1;859995728;3;-17806589;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1029;;1;859995732;3;-17824400;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1030;;1;859995736;3;-17842211;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1031;;1;859995740;3;-17860022;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1032;;1;859995744;3;-17877833;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1033;;1;859995748;3;-17895644;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1034;;1;859995752;3;-17913455;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1035;;1;859995756;3;-17931266;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1036;;1;859995760;3;-17949077;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1037;;1;859995764;3;-17966888;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1038;;1;859995768;3;-17984699;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1039;;1;859995772;3;-18002510;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1040;;1;859995776;3;-18020321;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1041;;1;859995780;3;-18038132;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1042;;1;859995784;3;-18055943;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1043;;1;859995788;3;-18073754;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1044;;1;859995792;3;-18091565;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1045;;1;859995796;3;-18109376;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1046;;1;859995800;3;-18127187;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1047;;1;859995804;3;-18144998;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1048;;1;859995808;3;-18162809;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1049;;1;859995812;3;-18180620;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1050;;1;859995816;3;-18198431;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1051;;1;859995820;3;-18216242;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1052;;1;859995824;3;-18234053;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1053;;1;859995828;3;-18251864;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1054;;1;859995832;3;-18269675;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1055;;1;859995836;3;-18287486;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1056;;1;859995840;3;-18305297;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1057;;1;859995844;3;-18323108;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1058;;1;859995848;3;-18340919;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1059;;1;859995852;3;-18358730;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1060;;1;859995856;3;-18376541;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1061;;1;859995860;3;-18394352;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1062;;1;859995864;3;-18412163;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1063;;1;859995868;3;-18429974;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1064;;1;859995872;3;-18447785;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1065;;1;859995876;3;-18465596;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1066;;1;859995880;3;-18483407;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1067;;1;859995884;3;-18501218;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1068;;1;859995888;3;-18519029;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1069;;1;859995892;3;-18536840;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1070;;1;859995896;3;-18554651;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1071;;1;859995900;3;-18572462;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1072;;1;859995904;3;-18590273;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1073;;1;859995908;3;-18608084;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1074;;1;859995912;3;-18625895;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1075;;1;859995916;3;-18643706;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1076;;1;859995920;3;-18661517;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1077;;1;859995924;3;-18679328;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1078;;1;859995928;3;-18697139;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1079;;1;859995932;3;-18714950;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1080;;1;859995936;3;-18732761;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1081;;1;859995940;3;-18750572;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1082;;1;859995944;3;-18768383;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1083;;1;859995948;3;-18786194;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1084;;1;859995952;3;-18804005;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1085;;1;859995956;3;-18821816;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1086;;1;859995960;3;-18839627;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1087;;1;859995964;3;-18857438;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1088;;1;859995968;3;-18875249;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1089;;1;859995972;3;-18893060;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1090;;1;859995976;3;-18910871;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1091;;1;859995980;3;-18928682;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1092;;1;859995984;3;-18946493;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1093;;1;859995988;3;-18964304;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1094;;1;859995992;3;-18982115;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1095;;1;859995996;3;-18999926;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1096;;1;859996000;3;-19017737;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1097;;1;859996004;3;-19035548;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1098;;1;859996008;3;-19053359;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1099;;1;859996012;3;-19071170;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1100;;1;859996016;3;-19088981;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1101;;1;859996020;3;-19106792;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1102;;1;859996024;3;-19124603;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1103;;1;859996028;3;-19142414;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1104;;1;859996032;3;-19160225;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1105;;1;859996036;3;-19178036;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1106;;1;859996040;3;-19195847;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1107;;1;859996044;3;-19213658;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1108;;1;859996048;3;-19231469;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1109;;1;859996052;3;-19249280;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1110;;1;859996056;3;-19267091;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1111;;1;859996060;3;-19284902;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1112;;1;859996064;3;-19302713;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1113;;1;859996068;3;-19320524;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1114;;1;859996072;3;-19338335;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1115;;1;859996076;3;-19356146;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1116;;1;859996080;3;-19373957;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1117;;1;859996084;3;-19391768;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1118;;1;859996088;3;-19409579;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1119;;1;859996092;3;-19427390;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1120;;1;859996096;3;-19445201;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1121;;1;859996100;3;-19463012;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1122;;1;859996104;3;-19480823;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1123;;1;859996108;3;-19498634;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1124;;1;859996112;3;-19516445;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1125;;1;859996116;3;-19534256;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1126;;1;859996120;3;-19552067;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1127;;1;859996124;3;-19569878;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1128;;1;859996128;3;-19587689;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1129;;1;859996132;3;-19605500;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1130;;1;859996136;3;-19623311;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1131;;1;859996140;3;-19641122;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1132;;1;859996144;3;-19658933;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1133;;1;859996148;3;-19676744;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1134;;1;859996152;3;-19694555;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1135;;1;859996156;3;-19712366;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1136;;1;859996160;3;-19730177;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1137;;1;859996164;3;-19747988;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1138;;1;859996168;3;-19765799;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1139;;1;859996172;3;-19783610;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1140;;1;859996176;3;-19801421;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1141;;1;859996180;3;-19819232;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1142;;1;859996184;3;-19837043;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1143;;1;859996188;3;-19854854;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1144;;1;859996192;3;-19872665;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1145;;1;859996196;3;-19890476;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1146;;1;859996200;3;-19908287;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1147;;1;859996204;3;-19926098;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1148;;1;859996208;3;-19943909;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1149;;1;859996212;3;-19961720;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1150;;1;859996216;3;-19979531;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1151;;1;859996220;3;-19997342;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1152;;1;859996224;3;-20015153;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1153;;1;859996228;3;-20032964;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1154;;1;859996232;3;-20050775;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1155;;1;859996236;3;-20068586;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1156;;1;859996240;3;-20086397;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1157;;1;859996244;3;-20104208;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1158;;1;859996248;3;-20122019;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1159;;1;859996252;3;-20139830;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1160;;1;859996256;3;-20157641;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1161;;1;859996260;3;-20175452;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1162;;1;859996264;3;-20193263;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1163;;1;859996268;3;-20211074;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1164;;1;859996272;3;-20228885;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1165;;1;859996276;3;-20246696;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1166;;1;859996280;3;-20264507;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1167;;1;859996284;3;-20282318;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1168;;1;859996288;3;-20300129;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1169;;1;859996292;3;-20317940;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1170;;1;859996296;3;-20335751;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1171;;1;859996300;3;-20353562;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1172;;1;859996304;3;-20371373;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1173;;1;859996308;3;-20389184;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1174;;1;859996312;3;-20406995;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1175;;1;859996316;3;-20424806;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1176;;1;859996320;3;-20442617;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1177;;1;859996324;3;-20460428;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1178;;1;859996328;3;-20478239;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1179;;1;859996332;3;-20496050;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1180;;1;859996336;3;-20513861;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1181;;1;859996340;3;-20531672;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1182;;1;859996344;3;-20549483;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1183;;1;859996348;3;-20567294;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1184;;1;859996352;3;-20585105;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1185;;1;859996356;3;-20602916;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1186;;1;859996360;3;-20620727;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1187;;1;859996364;3;-20638538;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1188;;1;859996368;3;-20656349;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1189;;1;859996372;3;-20674160;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1190;;1;859996376;3;-20691971;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1191;;1;859996380;3;-20709782;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1192;;1;859996384;3;-20727593;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1193;;1;859996388;3;-20745404;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1194;;1;859996392;3;-20763215;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1195;;1;859996396;3;-20781026;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1196;;1;859996400;3;-20798837;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1197;;1;859996404;3;-20816648;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1198;;1;859996408;3;-20834459;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1199;;1;859996412;3;-20852270;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1200;;1;859996416;3;-20870081;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1201;;1;859996420;3;-20887892;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1202;;1;859996424;3;-20905703;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1203;;1;859996428;3;-20923514;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1204;;1;859996432;3;-20941325;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1205;;1;859996436;3;-20959136;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1206;;1;859996440;3;-20976947;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1207;;1;859996444;3;-20994758;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1208;;1;859996448;3;-21012569;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1209;;1;859996452;3;-21030380;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1210;;1;859996456;3;-21048191;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1211;;1;859996460;3;-21066002;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1212;;1;859996464;3;-21083813;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1213;;1;859996468;3;-21101624;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1214;;1;859996472;3;-21119435;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1215;;1;859996476;3;-21137246;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1216;;1;859996480;3;-21155057;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1217;;1;859996484;3;-21172868;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1218;;1;859996488;3;-21190679;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1219;;1;859996492;3;-21208490;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1220;;1;859996496;3;-21226301;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1221;;1;859996500;3;-21244112;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1222;;1;859996504;3;-21261923;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1223;;1;859996508;3;-21279734;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1224;;1;859996512;3;-21297545;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1225;;1;859996516;3;-21315356;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1226;;1;859996520;3;-21333167;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1227;;1;859996524;3;-21350978;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1228;;1;859996528;3;-21368789;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1229;;1;859996532;3;-21386600;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1230;;1;859996536;3;-21404411;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1231;;1;859996540;3;-21422222;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1232;;1;859996544;3;-21440033;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1233;;1;859996548;3;-21457844;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1234;;1;859996552;3;-21475655;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1235;;1;859996556;3;-21493466;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1236;;1;859996560;3;-21511277;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1237;;1;859996564;3;-21529088;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1238;;1;859996568;3;-21546899;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1239;;1;859996572;3;-21564710;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1240;;1;859996576;3;-21582521;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1241;;1;859996580;3;-21600332;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1242;;1;859996584;3;-21618143;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1243;;1;859996588;3;-21635954;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1244;;1;859996592;3;-21653765;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1245;;1;859996596;3;-21671576;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1246;;1;859996600;3;-21689387;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1247;;1;859996604;3;-21707198;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1248;;1;859996608;3;-21725009;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1249;;1;859996612;3;-21742820;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1250;;1;859996616;3;-21760631;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1251;;1;859996620;3;-21778442;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1252;;1;859996624;3;-21796253;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1253;;1;859996628;3;-21814064;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1254;;1;859996632;3;-21831875;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1255;;1;859996636;3;-21849686;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1256;;1;859996640;3;-21867497;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1257;;1;859996644;3;-21885308;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1258;;1;859996648;3;-21903119;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1259;;1;859996652;3;-21920930;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1260;;1;859996656;3;-21938741;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1261;;1;859996660;3;-21956552;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1262;;1;859996664;3;-21974363;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1263;;1;859996668;3;-21992174;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1264;;1;859996672;3;-22009985;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1265;;1;859996676;3;-22027796;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1266;;1;859996680;3;-22045607;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1267;;1;859996684;3;-22063418;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1268;;1;859996688;3;-22081229;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1269;;1;859996692;3;-22099040;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1270;;1;859996696;3;-22116851;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1271;;1;859996700;3;-22134662;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1272;;1;859996704;3;-22152473;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1273;;1;859996708;3;-22170284;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1274;;1;859996712;3;-22188095;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1275;;1;859996716;3;-22205906;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1276;;1;859996720;3;-22223717;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1277;;1;859996724;3;-22241528;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1278;;1;859996728;3;-22259339;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1279;;1;859996732;3;-22277150;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1280;;1;859996736;3;-22294961;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1281;;1;859996740;3;-22312772;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1282;;1;859996744;3;-22330583;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1283;;1;859996748;3;-22348394;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1284;;1;859996752;3;-22366205;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1285;;1;859996756;3;-22384016;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1286;;1;859996760;3;-22401827;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1287;;1;859996764;3;-22419638;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1288;;1;859996768;3;-22437449;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1289;;1;859996772;3;-22455260;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1290;;1;859996776;3;-22473071;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1291;;1;859996780;3;-22490882;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1292;;1;859996784;3;-22508693;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1293;;1;859996788;3;-22526504;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1294;;1;859996792;3;-22544315;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1295;;1;859996796;3;-22562126;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1296;;1;859996800;3;-22579937;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1297;;1;859996804;3;-22597748;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1298;;1;859996808;3;-22615559;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1299;;1;859996812;3;-22633370;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1300;;1;859996816;3;-22651181;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1301;;1;859996820;3;-22668992;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1302;;1;859996824;3;-22686803;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1303;;1;859996828;3;-22704614;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1304;;1;859996832;3;-22722425;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1305;;1;859996836;3;-22740236;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1306;;1;859996840;3;-22758047;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1307;;1;859996844;3;-22775858;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1308;;1;859996848;3;-22793669;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1309;;1;859996852;3;-22811480;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1310;;1;859996856;3;-22829291;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1311;;1;859996860;3;-22847102;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1312;;1;859996864;3;-22864913;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1313;;1;859996868;3;-22882724;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1314;;1;859996872;3;-22900535;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1315;;1;859996876;3;-22918346;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1316;;1;859996880;3;-22936157;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1317;;1;859996884;3;-22953968;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1318;;1;859996888;3;-22971779;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1319;;1;859996892;3;-22989590;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1320;;1;859996896;3;-23007401;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1321;;1;859996900;3;-23025212;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1322;;1;859996904;3;-23043023;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1323;;1;859996908;3;-23060834;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1324;;1;859996912;3;-23078645;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1325;;1;859996916;3;-23096456;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1326;;1;859996920;3;-23114267;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1327;;1;859996924;3;-23132078;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1328;;1;859996928;3;-23149889;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1329;;1;859996932;3;-23167700;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1330;;1;859996936;3;-23185511;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1331;;1;859996940;3;-23203322;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1332;;1;859996944;3;-23221133;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1333;;1;859996948;3;-23238944;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1334;;1;859996952;3;-23256755;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1335;;1;859996956;3;-23274566;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1336;;1;859996960;3;-23292377;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1337;;1;859996964;3;-23310188;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1338;;1;859996968;3;-23327999;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1339;;1;859996972;3;-23345810;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1340;;1;859996976;3;-23363621;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1341;;1;859996980;3;-23381432;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1342;;1;859996984;3;-23399243;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1343;;1;859996988;3;-23417054;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1344;;1;859996992;3;-23434865;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1345;;1;859996996;3;-23452676;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1346;;1;859997000;3;-23470487;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1347;;1;859997004;3;-23488298;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1348;;1;859997008;3;-23506109;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1349;;1;859997012;3;-23523920;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1350;;1;859997016;3;-23541731;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1351;;1;859997020;3;-23559542;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1352;;1;859997024;3;-23577353;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1353;;1;859997028;3;-23595164;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1354;;1;859997032;3;-23612975;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1355;;1;859997036;3;-23630786;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1356;;1;859997040;3;-23648597;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1357;;1;859997044;3;-23666408;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1358;;1;859997048;3;-23684219;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1359;;1;859997052;3;-23702030;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1360;;1;859997056;3;-23719841;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1361;;1;859997060;3;-23737652;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1362;;1;859997064;3;-23755463;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1363;;1;859997068;3;-23773274;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1364;;1;859997072;3;-23791085;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1365;;1;859997076;3;-23808896;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1366;;1;859997080;3;-23826707;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1367;;1;859997084;3;-23844518;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1368;;1;859997088;3;-23862329;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1369;;1;859997092;3;-23880140;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1370;;1;859997096;3;-23897951;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1371;;1;859997100;3;-23915762;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1372;;1;859997104;3;-23933573;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1373;;1;859997108;3;-23951384;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1374;;1;859997112;3;-23969195;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1375;;1;859997116;3;-23987006;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1376;;1;859997120;3;-24004817;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1377;;1;859997124;3;-24022628;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1378;;1;859997128;3;-24040439;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1379;;1;859997132;3;-24058250;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1380;;1;859997136;3;-24076061;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1381;;1;859997140;3;-24093872;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1382;;1;859997144;3;-24111683;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1383;;1;859997148;3;-24129494;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1384;;1;859997152;3;-24147305;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1385;;1;859997156;3;-24165116;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1386;;1;859997160;3;-24182927;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1387;;1;859997164;3;-24200738;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1388;;1;859997168;3;-24218549;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1389;;1;859997172;3;-24236360;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1390;;1;859997176;3;-24254171;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1391;;1;859997180;3;-24271982;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1392;;1;859997184;3;-24289793;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1393;;1;859997188;3;-24307604;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1394;;1;859997192;3;-24325415;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1395;;1;859997196;3;-24343226;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1396;;1;859997200;3;-24361037;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1397;;1;859997204;3;-24378848;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1398;;1;859997208;3;-24396659;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1399;;1;859997212;3;-24414470;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1400;;1;859997216;3;-24432281;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1401;;1;859997220;3;-24450092;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1402;;1;859997224;3;-24467903;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1403;;1;859997228;3;-24485714;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1404;;1;859997232;3;-24503525;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1405;;1;859997236;3;-24521336;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1406;;1;859997240;3;-24539147;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1407;;1;859997244;3;-24556958;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1408;;1;859997248;3;-24574769;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1409;;1;859997252;3;-24592580;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1410;;1;859997256;3;-24610391;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1411;;1;859997260;3;-24628202;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1412;;1;859997264;3;-24646013;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1413;;1;859997268;3;-24663824;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1414;;1;859997272;3;-24681635;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1415;;1;859997276;3;-24699446;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1416;;1;859997280;3;-24717257;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1417;;1;859997284;3;-24735068;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1418;;1;859997288;3;-24752879;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1419;;1;859997292;3;-24770690;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1420;;1;859997296;3;-24788501;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1421;;1;859997300;3;-24806312;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1422;;1;859997304;3;-24824123;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1423;;1;859997308;3;-24841934;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1424;;1;859997312;3;-24859745;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1425;;1;859997316;3;-24877556;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1426;;1;859997320;3;-24895367;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1427;;1;859997324;3;-24913178;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1428;;1;859997328;3;-24930989;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1429;;1;859997332;3;-24948800;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1430;;1;859997336;3;-24966611;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1431;;1;859997340;3;-24984422;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1432;;1;859997344;3;-25002233;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1433;;1;859997348;3;-25020044;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1434;;1;859997352;3;-25037855;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1435;;1;859997356;3;-25055666;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1436;;1;859997360;3;-25073477;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1437;;1;859997364;3;-25091288;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1438;;1;859997368;3;-25109099;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1439;;1;859997372;3;-25126910;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1440;;1;859997376;3;-25144721;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1441;;1;859997380;3;-25162532;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1442;;1;859997384;3;-25180343;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1443;;1;859997388;3;-25198154;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1444;;1;859997392;3;-25215965;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1445;;1;859997396;3;-25233776;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1446;;1;859997400;3;-25251587;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1447;;1;859997404;3;-25269398;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1448;;1;859997408;3;-25287209;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1449;;1;859997412;3;-25305020;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1450;;1;859997416;3;-25322831;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1451;;1;859997420;3;-25340642;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1452;;1;859997424;3;-25358453;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1453;;1;859997428;3;-25376264;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1454;;1;859997432;3;-25394075;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1455;;1;859997436;3;-25411886;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1456;;1;859997440;3;-25429697;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1457;;1;859997444;3;-25447508;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1458;;1;859997448;3;-25465319;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1459;;1;859997452;3;-25483130;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1460;;1;859997456;3;-25500941;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1461;;1;859997460;3;-25518752;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1462;;1;859997464;3;-25536563;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1463;;1;859997468;3;-25554374;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1464;;1;859997472;3;-25572185;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1465;;1;859997476;3;-25589996;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1466;;1;859997480;3;-25607807;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1467;;1;859997484;3;-25625618;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1468;;1;859997488;3;-25643429;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1469;;1;859997492;3;-25661240;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1470;;1;859997496;3;-25679051;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1471;;1;859997500;3;-25696862;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1472;;1;859997504;3;-25714673;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1473;;1;859997508;3;-25732484;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1474;;1;859997512;3;-25750295;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1475;;1;859997516;3;-25768106;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1476;;1;859997520;3;-25785917;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1477;;1;859997524;3;-25803728;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1478;;1;859997528;3;-25821539;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1479;;1;859997532;3;-25839350;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1480;;1;859997536;3;-25857161;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1481;;1;859997540;3;-25874972;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1482;;1;859997544;3;-25892783;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1483;;1;859997548;3;-25910594;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1484;;1;859997552;3;-25928405;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1485;;1;859997556;3;-25946216;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1486;;1;859997560;3;-25964027;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1487;;1;859997564;3;-25981838;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1488;;1;859997568;3;-25999649;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1489;;1;859997572;3;-26017460;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1490;;1;859997576;3;-26035271;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1491;;1;859997580;3;-26053082;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1492;;1;859997584;3;-26070893;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1493;;1;859997588;3;-26088704;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1494;;1;859997592;3;-26106515;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1495;;1;859997596;3;-26124326;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1496;;1;859997600;3;-26142137;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1497;;1;859997604;3;-26159948;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1498;;1;859997608;3;-26177759;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1499;;1;859997612;3;-26195570;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1500;;1;859997616;3;-26213381;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1501;;1;859997620;3;-26231192;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1502;;1;859997624;3;-26249003;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1503;;1;859997628;3;-26266814;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1504;;1;859997632;3;-26284625;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1505;;1;859997636;3;-26302436;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1506;;1;859997640;3;-26320247;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1507;;1;859997644;3;-26338058;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1508;;1;859997648;3;-26355869;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1509;;1;859997652;3;-26373680;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1510;;1;859997656;3;-26391491;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1511;;1;859997660;3;-26409302;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1512;;1;859997664;3;-26427113;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1513;;1;859997668;3;-26444924;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1514;;1;859997672;3;-26462735;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1515;;1;859997676;3;-26480546;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1516;;1;859997680;3;-26498357;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1517;;1;859997684;3;-26516168;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1518;;1;859997688;3;-26533979;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1519;;1;859997692;3;-26551790;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1520;;1;859997696;3;-26569601;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1521;;1;859997700;3;-26587412;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1522;;1;859997704;3;-26605223;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1523;;1;859997708;3;-26623034;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1524;;1;859997712;3;-26640845;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1525;;1;859997716;3;-26658656;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1526;;1;859997720;3;-26676467;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1527;;1;859997724;3;-26694278;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1528;;1;859997728;3;-26712089;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1529;;1;859997732;3;-26729900;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1530;;1;859997736;3;-26747711;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1531;;1;859997740;3;-26765522;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1532;;1;859997744;3;-26783333;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1533;;1;859997748;3;-26801144;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1534;;1;859997752;3;-26818955;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1535;;1;859997756;3;-26836766;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1536;;1;859997760;3;-26854577;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1537;;1;859997764;3;-26872388;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1538;;1;859997768;3;-26890199;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1539;;1;859997772;3;-26908010;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1540;;1;859997776;3;-26925821;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1541;;1;859997780;3;-26943632;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1542;;1;859997784;3;-26961443;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1543;;1;859997788;3;-26979254;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1544;;1;859997792;3;-26997065;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1545;;1;859997796;3;-27014876;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1546;;1;859997800;3;-27032687;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1547;;1;859997804;3;-27050498;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1548;;1;859997808;3;-27068309;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1549;;1;859997812;3;-27086120;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1550;;1;859997816;3;-27103931;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1551;;1;859997820;3;-27121742;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1552;;1;859997824;3;-27139553;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1553;;1;859997828;3;-27157364;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1554;;1;859997832;3;-27175175;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1555;;1;859997836;3;-27192986;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1556;;1;859997840;3;-27210797;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1557;;1;859997844;3;-27228608;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1558;;1;859997848;3;-27246419;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1559;;1;859997852;3;-27264230;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1560;;1;859997856;3;-27282041;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1561;;1;859997860;3;-27299852;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1562;;1;859997864;3;-27317663;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1563;;1;859997868;3;-27335474;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1564;;1;859997872;3;-27353285;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1565;;1;859997876;3;-27371096;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1566;;1;859997880;3;-27388907;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1567;;1;859997884;3;-27406718;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1568;;1;859997888;3;-27424529;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1569;;1;859997892;3;-27442340;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1570;;1;859997896;3;-27460151;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1571;;1;859997900;3;-27477962;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1572;;1;859997904;3;-27495773;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1573;;1;859997908;3;-27513584;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1574;;1;859997912;3;-27531395;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1575;;1;859997916;3;-27549206;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1576;;1;859997920;3;-27567017;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1577;;1;859997924;3;-27584828;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1578;;1;859997928;3;-27602639;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1579;;1;859997932;3;-27620450;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1580;;1;859997936;3;-27638261;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1581;;1;859997940;3;-27656072;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1582;;1;859997944;3;-27673883;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1583;;1;859997948;3;-27691694;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1584;;1;859997952;3;-27709505;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1585;;1;859997956;3;-27727316;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1586;;1;859997960;3;-27745127;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1587;;1;859997964;3;-27762938;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1588;;1;859997968;3;-27780749;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1589;;1;859997972;3;-27798560;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1590;;1;859997976;3;-27816371;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1591;;1;859997980;3;-27834182;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1592;;1;859997984;3;-27851993;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1593;;1;859997988;3;-27869804;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1594;;1;859997992;3;-27887615;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1595;;1;859997996;3;-27905426;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1596;;1;859998000;3;-27923237;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1597;;1;859998004;3;-27941048;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1598;;1;859998008;3;-27958859;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1599;;1;859998012;3;-27976670;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1600;;1;859998016;3;-27994481;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1601;;1;859998020;3;-28012292;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1602;;1;859998024;3;-28030103;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1603;;1;859998028;3;-28047914;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1604;;1;859998032;3;-28065725;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1605;;1;859998036;3;-28083536;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1606;;1;859998040;3;-28101347;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1607;;1;859998044;3;-28119158;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1608;;1;859998048;3;-28136969;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1609;;1;859998052;3;-28154780;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1610;;1;859998056;3;-28172591;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1611;;1;859998060;3;-28190402;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1612;;1;859998064;3;-28208213;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1613;;1;859998068;3;-28226024;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1614;;1;859998072;3;-28243835;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1615;;1;859998076;3;-28261646;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1616;;1;859998080;3;-28279457;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1617;;1;859998084;3;-28297268;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1618;;1;859998088;3;-28315079;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1619;;1;859998092;3;-28332890;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1620;;1;859998096;3;-28350701;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1621;;1;859998100;3;-28368512;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1622;;1;859998104;3;-28386323;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1623;;1;859998108;3;-28404134;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1624;;1;859998112;3;-28421945;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1625;;1;859998116;3;-28439756;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1626;;1;859998120;3;-28457567;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1627;;1;859998124;3;-28475378;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1628;;1;859998128;3;-28493189;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1629;;1;859998132;3;-28511000;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1630;;1;859998136;3;-28528811;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1631;;1;859998140;3;-28546622;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1632;;1;859998144;3;-28564433;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1633;;1;859998148;3;-28582244;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1634;;1;859998152;3;-28600055;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1635;;1;859998156;3;-28617866;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1636;;1;859998160;3;-28635677;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1637;;1;859998164;3;-28653488;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1638;;1;859998168;3;-28671299;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1639;;1;859998172;3;-28689110;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1640;;1;859998176;3;-28706921;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1641;;1;859998180;3;-28724732;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1642;;1;859998184;3;-28742543;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1643;;1;859998188;3;-28760354;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1644;;1;859998192;3;-28778165;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1645;;1;859998196;3;-28795976;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1646;;1;859998200;3;-28813787;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1647;;1;859998204;3;-28831598;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1648;;1;859998208;3;-28849409;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1649;;1;859998212;3;-28867220;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1650;;1;859998216;3;-28885031;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1651;;1;859998220;3;-28902842;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1652;;1;859998224;3;-28920653;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1653;;1;859998228;3;-28938464;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1654;;1;859998232;3;-28956275;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1655;;1;859998236;3;-28974086;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1656;;1;859998240;3;-28991897;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1657;;1;859998244;3;-29009708;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1658;;1;859998248;3;-29027519;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1659;;1;859998252;3;-29045330;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1660;;1;859998256;3;-29063141;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1661;;1;859998260;3;-29080952;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1662;;1;859998264;3;-29098763;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1663;;1;859998268;3;-29116574;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1664;;1;859998272;3;-29134385;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1665;;1;859998276;3;-29152196;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1666;;1;859998280;3;-29170007;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1667;;1;859998284;3;-29187818;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1668;;1;859998288;3;-29205629;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1669;;1;859998292;3;-29223440;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1670;;1;859998296;3;-29241251;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1671;;1;859998300;3;-29259062;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1672;;1;859998304;3;-29276873;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1673;;1;859998308;3;-29294684;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1674;;1;859998312;3;-29312495;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1675;;1;859998316;3;-29330306;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1676;;1;859998320;3;-29348117;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1677;;1;859998324;3;-29365928;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1678;;1;859998328;3;-29383739;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1679;;1;859998332;3;-29401550;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1680;;1;859998336;3;-29419361;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1681;;1;859998340;3;-29437172;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1682;;1;859998344;3;-29454983;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1683;;1;859998348;3;-29472794;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1684;;1;859998352;3;-29490605;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1685;;1;859998356;3;-29508416;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1686;;1;859998360;3;-29526227;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1687;;1;859998364;3;-29544038;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1688;;1;859998368;3;-29561849;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1689;;1;859998372;3;-29579660;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1690;;1;859998376;3;-29597471;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1691;;1;859998380;3;-29615282;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1692;;1;859998384;3;-29633093;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1693;;1;859998388;3;-29650904;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1694;;1;859998392;3;-29668715;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1695;;1;859998396;3;-29686526;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1696;;1;859998400;3;-29704337;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1697;;1;859998404;3;-29722148;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1698;;1;859998408;3;-29739959;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1699;;1;859998412;3;-29757770;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1700;;1;859998416;3;-29775581;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1701;;1;859998420;3;-29793392;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1702;;1;859998424;3;-29811203;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1703;;1;859998428;3;-29829014;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1704;;1;859998432;3;-29846825;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1705;;1;859998436;3;-29864636;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1706;;1;859998440;3;-29882447;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1707;;1;859998444;3;-29900258;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1708;;1;859998448;3;-29918069;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1709;;1;859998452;3;-29935880;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1710;;1;859998456;3;-29953691;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1711;;1;859998460;3;-29971502;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1712;;1;859998464;3;-29989313;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1713;;1;859998468;3;-30007124;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1714;;1;859998472;3;-30024935;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1715;;1;859998476;3;-30042746;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1716;;1;859998480;3;-30060557;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1717;;1;859998484;3;-30078368;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1718;;1;859998488;3;-30096179;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1719;;1;859998492;3;-30113990;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1720;;1;859998496;3;-30131801;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1721;;1;859998500;3;-30149612;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1722;;1;859998504;3;-30167423;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1723;;1;859998508;3;-30185234;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1724;;1;859998512;3;-30203045;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1725;;1;859998516;3;-30220856;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1726;;1;859998520;3;-30238667;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1727;;1;859998524;3;-30256478;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1728;;1;859998528;3;-30274289;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1729;;1;859998532;3;-30292100;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1730;;1;859998536;3;-30309911;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1731;;1;859998540;3;-30327722;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1732;;1;859998544;3;-30345533;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1733;;1;859998548;3;-30363344;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1734;;1;859998552;3;-30381155;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1735;;1;859998556;3;-30398966;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1736;;1;859998560;3;-30416777;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1737;;1;859998564;3;-30434588;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1738;;1;859998568;3;-30452399;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1739;;1;859998572;3;-30470210;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1740;;1;859998576;3;-30488021;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1741;;1;859998580;3;-30505832;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1742;;1;859998584;3;-30523643;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1743;;1;859998588;3;-30541454;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1744;;1;859998592;3;-30559265;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1745;;1;859998596;3;-30577076;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1746;;1;859998600;3;-30594887;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1747;;1;859998604;3;-30612698;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1748;;1;859998608;3;-30630509;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1749;;1;859998612;3;-30648320;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1750;;1;859998616;3;-30666131;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1751;;1;859998620;3;-30683942;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1752;;1;859998624;3;-30701753;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1753;;1;859998628;3;-30719564;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1754;;1;859998632;3;-30737375;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1755;;1;859998636;3;-30755186;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1756;;1;859998640;3;-30772997;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1757;;1;859998644;3;-30790808;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1758;;1;859998648;3;-30808619;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1759;;1;859998652;3;-30826430;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1760;;1;859998656;3;-30844241;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1761;;1;859998660;3;-30862052;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1762;;1;859998664;3;-30879863;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1763;;1;859998668;3;-30897674;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1764;;1;859998672;3;-30915485;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1765;;1;859998676;3;-30933296;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1766;;1;859998680;3;-30951107;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1767;;1;859998684;3;-30968918;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1768;;1;859998688;3;-30986729;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1769;;1;859998692;3;-31004540;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1770;;1;859998696;3;-31022351;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1771;;1;859998700;3;-31040162;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1772;;1;859998704;3;-31057973;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1773;;1;859998708;3;-31075784;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1774;;1;859998712;3;-31093595;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1775;;1;859998716;3;-31111406;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1776;;1;859998720;3;-31129217;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1777;;1;859998724;3;-31147028;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1778;;1;859998728;3;-31164839;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1779;;1;859998732;3;-31182650;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1780;;1;859998736;3;-31200461;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1781;;1;859998740;3;-31218272;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1782;;1;859998744;3;-31236083;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1783;;1;859998748;3;-31253894;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1784;;1;859998752;3;-31271705;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1785;;1;859998756;3;-31289516;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1786;;1;859998760;3;-31307327;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1787;;1;859998764;3;-31325138;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1788;;1;859998768;3;-31342949;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1789;;1;859998772;3;-31360760;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1790;;1;859998776;3;-31378571;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1791;;1;859998780;3;-31396382;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1792;;1;859998784;3;-31414193;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1793;;1;859998788;3;-31432004;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1794;;1;859998792;3;-31449815;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1795;;1;859998796;3;-31467626;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1796;;1;859998800;3;-31485437;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1797;;1;859998804;3;-31503248;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1798;;1;859998808;3;-31521059;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1799;;1;859998812;3;-31538870;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1800;;1;859998816;3;-31556681;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1801;;1;859998820;3;-31574492;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1802;;1;859998824;3;-31592303;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1803;;1;859998828;3;-31610114;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1804;;1;859998832;3;-31627925;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1805;;1;859998836;3;-31645736;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1806;;1;859998840;3;-31663547;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1807;;1;859998844;3;-31681358;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1808;;1;859998848;3;-31699169;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1809;;1;859998852;3;-31716980;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1810;;1;859998856;3;-31734791;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1811;;1;859998860;3;-31752602;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1812;;1;859998864;3;-31770413;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1813;;1;859998868;3;-31788224;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1814;;1;859998872;3;-31806035;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1815;;1;859998876;3;-31823846;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1816;;1;859998880;3;-31841657;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1817;;1;859998884;3;-31859468;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1818;;1;859998888;3;-31877279;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1819;;1;859998892;3;-31895090;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1820;;1;859998896;3;-31912901;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1821;;1;859998900;3;-31930712;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1822;;1;859998904;3;-31948523;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1823;;1;859998908;3;-31966334;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1824;;1;859998912;3;-31984145;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1825;;1;859998916;3;-32001956;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1826;;1;859998920;3;-32019767;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1827;;1;859998924;3;-32037578;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1828;;1;859998928;3;-32055389;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1829;;1;859998932;3;-32073200;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1830;;1;859998936;3;-32091011;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1831;;1;859998940;3;-32108822;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1832;;1;859998944;3;-32126633;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1833;;1;859998948;3;-32144444;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1834;;1;859998952;3;-32162255;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1835;;1;859998956;3;-32180066;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1836;;1;859998960;3;-32197877;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1837;;1;859998964;3;-32215688;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1838;;1;859998968;3;-32233499;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1839;;1;859998972;3;-32251310;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1840;;1;859998976;3;-32269121;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1841;;1;859998980;3;-32286932;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1842;;1;859998984;3;-32304743;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1843;;1;859998988;3;-32322554;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1844;;1;859998992;3;-32340365;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1845;;1;859998996;3;-32358176;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1846;;1;859999000;3;-32375987;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1847;;1;859999004;3;-32393798;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1848;;1;859999008;3;-32411609;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1849;;1;859999012;3;-32429420;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1850;;1;859999016;3;-32447231;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1851;;1;859999020;3;-32465042;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1852;;1;859999024;3;-32482853;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1853;;1;859999028;3;-32500664;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1854;;1;859999032;3;-32518475;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1855;;1;859999036;3;-32536286;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1856;;1;859999040;3;-32554097;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1857;;1;859999044;3;-32571908;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1858;;1;859999048;3;-32589719;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1859;;1;859999052;3;-32607530;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1860;;1;859999056;3;-32625341;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1861;;1;859999060;3;-32643152;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1862;;1;859999064;3;-32660963;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1863;;1;859999068;3;-32678774;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1864;;1;859999072;3;-32696585;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1865;;1;859999076;3;-32714396;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1866;;1;859999080;3;-32732207;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1867;;1;859999084;3;-32750018;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1868;;1;859999088;3;-32767829;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1869;;1;859999092;3;-32785640;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1870;;1;859999096;3;-32803451;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1871;;1;859999100;3;-32821262;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1872;;1;859999104;3;-32839073;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1873;;1;859999108;3;-32856884;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1874;;1;859999112;3;-32874695;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1875;;1;859999116;3;-32892506;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1876;;1;859999120;3;-32910317;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1877;;1;859999124;3;-32928128;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1878;;1;859999128;3;-32945939;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1879;;1;859999132;3;-32963750;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1880;;1;859999136;3;-32981561;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1881;;1;859999140;3;-32999372;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1882;;1;859999144;3;-33017183;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1883;;1;859999148;3;-33034994;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1884;;1;859999152;3;-33052805;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1885;;1;859999156;3;-33070616;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1886;;1;859999160;3;-33088427;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1887;;1;859999164;3;-33106238;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1888;;1;859999168;3;-33124049;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1889;;1;859999172;3;-33141860;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1890;;1;859999176;3;-33159671;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1891;;1;859999180;3;-33177482;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1892;;1;859999184;3;-33195293;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1893;;1;859999188;3;-33213104;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1894;;1;859999192;3;-33230915;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1895;;1;859999196;3;-33248726;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1896;;1;859999200;3;-33266537;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1897;;1;859999204;3;-33284348;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1898;;1;859999208;3;-33302159;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1899;;1;859999212;3;-33319970;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1900;;1;859999216;3;-33337781;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1901;;1;859999220;3;-33355592;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1902;;1;859999224;3;-33373403;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1903;;1;859999228;3;-33391214;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1904;;1;859999232;3;-33409025;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1905;;1;859999236;3;-33426836;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1906;;1;859999240;3;-33444647;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1907;;1;859999244;3;-33462458;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1908;;1;859999248;3;-33480269;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1909;;1;859999252;3;-33498080;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1910;;1;859999256;3;-33515891;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1911;;1;859999260;3;-33533702;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1912;;1;859999264;3;-33551513;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1913;;1;859999268;3;-33569324;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1914;;1;859999272;3;-33587135;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1915;;1;859999276;3;-33604946;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1916;;1;859999280;3;-33622757;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1917;;1;859999284;3;-33640568;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1918;;1;859999288;3;-33658379;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1919;;1;859999292;3;-33676190;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1920;;1;859999296;3;-33694001;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1921;;1;859999300;3;-33711812;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1922;;1;859999304;3;-33729623;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1923;;1;859999308;3;-33747434;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1924;;1;859999312;3;-33765245;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1925;;1;859999316;3;-33783056;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1926;;1;859999320;3;-33800867;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1927;;1;859999324;3;-33818678;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1928;;1;859999328;3;-33836489;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1929;;1;859999332;3;-33854300;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1930;;1;859999336;3;-33872111;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1931;;1;859999340;3;-33889922;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1932;;1;859999344;3;-33907733;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1933;;1;859999348;3;-33925544;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1934;;1;859999352;3;-33943355;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1935;;1;859999356;3;-33961166;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1936;;1;859999360;3;-33978977;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1937;;1;859999364;3;-33996788;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1938;;1;859999368;3;-34014599;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1939;;1;859999372;3;-34032410;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1940;;1;859999376;3;-34050221;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1941;;1;859999380;3;-34068032;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1942;;1;859999384;3;-34085843;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1943;;1;859999388;3;-34103654;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1944;;1;859999392;3;-34121465;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1945;;1;859999396;3;-34139276;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1946;;1;859999400;3;-34157087;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1947;;1;859999404;3;-34174898;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1948;;1;859999408;3;-34192709;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1949;;1;859999412;3;-34210520;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1950;;1;859999416;3;-34228331;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1951;;1;859999420;3;-34246142;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1952;;1;859999424;3;-34263953;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1953;;1;859999428;3;-34281764;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1954;;1;859999432;3;-34299575;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1955;;1;859999436;3;-34317386;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1956;;1;859999440;3;-34335197;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1957;;1;859999444;3;-34353008;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1958;;1;859999448;3;-34370819;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1959;;1;859999452;3;-34388630;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1960;;1;859999456;3;-34406441;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1961;;1;859999460;3;-34424252;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1962;;1;859999464;3;-34442063;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1963;;1;859999468;3;-34459874;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1964;;1;859999472;3;-34477685;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1965;;1;859999476;3;-34495496;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1966;;1;859999480;3;-34513307;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1967;;1;859999484;3;-34531118;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1968;;1;859999488;3;-34548929;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1969;;1;859999492;3;-34566740;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1970;;1;859999496;3;-34584551;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1971;;1;859999500;3;-34602362;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1972;;1;859999504;3;-34620173;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1973;;1;859999508;3;-34637984;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1974;;1;859999512;3;-34655795;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1975;;1;859999516;3;-34673606;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1976;;1;859999520;3;-34691417;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1977;;1;859999524;3;-34709228;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1978;;1;859999528;3;-34727039;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1979;;1;859999532;3;-34744850;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1980;;1;859999536;3;-34762661;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1981;;1;859999540;3;-34780472;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1982;;1;859999544;3;-34798283;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1983;;1;859999548;3;-34816094;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1984;;1;859999552;3;-34833905;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1985;;1;859999556;3;-34851716;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1986;;1;859999560;3;-34869527;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1987;;1;859999564;3;-34887338;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1988;;1;859999568;3;-34905149;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1989;;1;859999572;3;-34922960;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1990;;1;859999576;3;-34940771;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1991;;1;859999580;3;-34958582;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1992;;1;859999584;3;-34976393;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1993;;1;859999588;3;-34994204;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1994;;1;859999592;3;-35012015;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1995;;1;859999596;3;-35029826;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1996;;1;859999600;3;-35047637;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1997;;1;859999604;3;-35065448;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1998;;1;859999608;3;-35083259;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d1999;;1;859999612;3;-35101070;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2000;;1;859999616;3;-35118881;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2001;;1;859999620;3;-35136692;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2002;;1;859999624;3;-35154503;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2003;;1;859999628;3;-35172314;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2004;;1;859999632;3;-35190125;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2005;;1;859999636;3;-35207936;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2006;;1;859999640;3;-35225747;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2007;;1;859999644;3;-35243558;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2008;;1;859999648;3;-35261369;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2009;;1;859999652;3;-35279180;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2010;;1;859999656;3;-35296991;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2011;;1;859999660;3;-35314802;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2012;;1;859999664;3;-35332613;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2013;;1;859999668;3;-35350424;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2014;;1;859999672;3;-35368235;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2015;;1;859999676;3;-35386046;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2016;;1;859999680;3;-35403857;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2017;;1;859999684;3;-35421668;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2018;;1;859999688;3;-35439479;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2019;;1;859999692;3;-35457290;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2020;;1;859999696;3;-35475101;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2021;;1;859999700;3;-35492912;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2022;;1;859999704;3;-35510723;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2023;;1;859999708;3;-35528534;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2024;;1;859999712;3;-35546345;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2025;;1;859999716;3;-35564156;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2026;;1;859999720;3;-35581967;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2027;;1;859999724;3;-35599778;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2028;;1;859999728;3;-35617589;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2029;;1;859999732;3;-35635400;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2030;;1;859999736;3;-35653211;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2031;;1;859999740;3;-35671022;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2032;;1;859999744;3;-35688833;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2033;;1;859999748;3;-35706644;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2034;;1;859999752;3;-35724455;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2035;;1;859999756;3;-35742266;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2036;;1;859999760;3;-35760077;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2037;;1;859999764;3;-35777888;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2038;;1;859999768;3;-35795699;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2039;;1;859999772;3;-35813510;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2040;;1;859999776;3;-35831321;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2041;;1;859999780;3;-35849132;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2042;;1;859999784;3;-35866943;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2043;;1;859999788;3;-35884754;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2044;;1;859999792;3;-35902565;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2045;;1;859999796;3;-35920376;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2046;;1;859999800;3;-35938187;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2047;;1;859999804;3;-35955998;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2048;;1;859999808;3;-35973809;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2049;;1;859999812;3;-35991620;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2050;;1;859999816;3;-36009431;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2051;;1;859999820;3;-36027242;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2052;;1;859999824;3;-36045053;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2053;;1;859999828;3;-36062864;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2054;;1;859999832;3;-36080675;1;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2055;;1;859999836;3;-36098486;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2056;;1;859999840;3;-36116297;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2057;;1;859999844;3;-36134108;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2058;;1;859999848;3;-36151919;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2059;;1;859999852;3;-36169730;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2060;;1;859999856;3;-36187541;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2061;;1;859999860;3;-36205352;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2062;;1;859999864;3;-36223163;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2063;;1;859999868;3;-36240974;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2064;;1;859999872;3;-36258785;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2065;;1;859999876;3;-36276596;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2066;;1;859999880;3;-36294407;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2067;;1;859999884;3;-36312218;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2068;;1;859999888;3;-36330029;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2069;;1;859999892;3;-36347840;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2070;;1;859999896;3;-36365651;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2071;;1;859999900;3;-36383462;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2072;;1;859999904;3;-36401273;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2073;;1;859999908;3;-36419084;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2074;;1;859999912;3;-36436895;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2075;;1;859999916;3;-36454706;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2076;;1;859999920;3;-36472517;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2077;;1;859999924;3;-36490328;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2078;;1;859999928;3;-36508139;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2079;;1;859999932;3;-36525950;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2080;;1;859999936;3;-36543761;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2081;;1;859999940;3;-36561572;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2082;;1;859999944;3;-36579383;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2083;;1;859999948;3;-36597194;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2084;;1;859999952;3;-36615005;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2085;;1;859999956;3;-36632816;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2086;;1;859999960;3;-36650627;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2087;;1;859999964;3;-36668438;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2088;;1;859999968;3;-36686249;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2089;;1;859999972;3;-36704060;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2090;;1;859999976;3;-36721871;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2091;;1;859999980;3;-36739682;4;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2092;;1;859999984;3;-36757493;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2093;;1;859999988;3;-36775304;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2094;;1;859999992;3;-36793115;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2095;;1;859999996;3;-36810926;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2096;;1;860000000;3;-36828737;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2097;;1;860000004;3;-36846548;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2098;;1;860000008;3;-36864359;8;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2099;;1;860000012;3;-36882170;3;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2100;;1;860000016;3;-36899981;7;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2101;;1;860000020;3;-36917792;5;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2102;;1;860000024;3;-36935603;2;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2103;;1;860000028;3;-36953414;9;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2104;;1;860000032;3;-36971225;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2105;;1;860000036;3;-36989036;6;br.com.itau.depositos.conta_criada
00263ce9-5bc3-3c6a-9933-efe63d58d2106;;1;860000040;3;-37006847;6;br.com.itau.depositos.conta_criada
